Wanted to share the logic of my best bot on the leaderboard so far. Till date, I'm still using Bayesian UCB as the base code and I added the following functions to make it better. I will briefly outline my strat here with a focus on the code, but will go into greater detail [here](https://www.kaggle.com/c/santa-2020/discussion/216458) about my various experiences with the different additions.
(In descending order of impact)

1. A logistic function that takes into account enemy action (1/(1+e^x) (This is within the action() function)
2. A function that tries to score how competent the enemy is, and therefore how much to perform (1). (This is in enemy_competence function)
3. Varying variance based on my score performance (This is in the get_variance function). 
4. A method to detect whether the enemy is using a simple logic to follow me (This is in the detect_follower function)
5. Explorer function to bump up exploration for the first 100 or so moves but vanishes after I/opponent has explored those machines (This is in explorer function)


In [ ]:

import numpy as np
from scipy.stats import beta
import random
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

class BayesianSpy():
    def __init__(self):
        self.post_a = [None]
        self.post_b = [None]
        self.bandit = None
        self.score_track = []
        self.player_track = []
        self.enemy_track = []
        self.total_reward = 0
        self.init_follow = False
        self.second_choices = 0
        self.cont_second_choices = 0
        self.post_a_track = []
        self.post_b_track = []
        self.increase_counter = 0
    
    def __call__(self, observation, configuration):
        if observation.step == 0:
            n_bandits = configuration.banditCount
            self.post_a = np.ones(n_bandits)
            self.post_b = np.ones(n_bandits)
        else:
            current_score= observation.reward
            r = current_score - self.total_reward
            player = observation.agentIndex
            enemy = 1-player
            self.total_reward = current_score
            self.score_track.append(current_score)
            self.enemy_track.append(observation.lastActions[enemy])
            self.player_track.append(observation.lastActions[player])
            # Update Gaussian posterior
            self.post_a[self.bandit] += r
            self.post_b[self.bandit] += 1 - r
            self.post_a_track.append(r)
            self.post_b_track.append(1-r)
        return self.action(observation, configuration)


    def random_top_choice(self, choices, std_limit = 0.0):
        std_dev = np.std(choices)
        lower = np.max(choices) - std_limit * std_dev
        shortlist = np.where(choices >= lower)[0]
    #     print(std_limit * std_dev)
    #     print(shortlist)
        return int(random.choice(shortlist))
    
    def second_choice(self, choices):
        choices_temp = choices.copy()
        choices_temp[np.argmax(choices_temp)] = -99
        return int(np.argmax(choices_temp))
    
    
    def detect_follower(self, lookback = 3, window = 50):
        follow_counts = 0
        repeat_counts = 0
        p = self.player_track[-window:]
        e = self.enemy_track[-window:]
        
        for i in range(lookback-1, len(p)-1):
            if p[i] in p[i-2:i] and p[i] != e[i-1]:
                #if i'm repeating/alternating but not by copying opponent
                repeat_counts +=1
                if e[i+1] in p[i-lookback+1:i+1]:
                    #opponent is copying me but not vice versa
                    follow_counts += 1
                    
        repeat_counts = max([repeat_counts,1])
        follow_score = follow_counts/repeat_counts
        
        return follow_score

    def alternate_choice(self, top_choice, follow_score, bound):
        if follow_score >= 0.5:
            self.init_follow = True
        elif follow_score <= 0.25:# turn off follow
            self.init_follow = False
            self.cont_second_choices = 0 #reset counter
    
    # firstly, only alternate if my top choice is same as last    
        is_last_2_same = (top_choice == self.player_track[-1]) #and (top_choice == p_track[-2])
        # secondly, only alternate if my top choice is not already being farmed by enemy
        not_circular_loop = top_choice not in self.enemy_track[-2:]
        if self.init_follow and is_last_2_same and not_circular_loop:
            bandit = self.second_choice(bound)
            self.second_choices += 1
            self.cont_second_choices += 1
        else: 
            bandit = top_choice
        return bandit

    def create_hist(self, track, n_bandits = 100):
        hist = [0] * n_bandits
        for k in track:
            hist[k] += 1
        return np.array(hist)
    
    def create_hist_redraws(self, track, lookback=3, n_bandits = 100):
        hist = [0] * n_bandits
        for i, k in enumerate(track):
            if i>=lookback and k in track[i-lookback:i]:
                hist[k] += 1
        return hist
    
    def unexplored_bandits(self,n_bandits = 100):
        unexplored = np.array([1] * n_bandits)
        for k in list(set(self.enemy_track)):
            unexplored[k] = 0.3
        for l in list(set(self.player_track)):
            unexplored[l] = 0
        return unexplored

    def enemy_competence(self, window = 100, lookback = 5, default = 0.3, lower_bound = 0.05):
        
        en_post_a = 1.0
        en_post_b = 1.0
        p_post_a = 1.0
        p_post_b = 1.0
        if len(self.enemy_track) < window:
            return default
        
        for i in range(1, min([len(self.player_track), window+1])):
            upper_window = min([-i + lookback, -1])
            lower_window = -i - lookback
            
            if self.player_track[-i] in self.enemy_track[lower_window:upper_window]:
                en_post_a += self.post_a_track[-i]
                en_post_b += self.post_b_track[-i]
            else: 
                p_post_a += self.post_a_track[-i]
                p_post_b += self.post_b_track[-i]
        if en_post_a + en_post_b <= 10 or p_post_a + p_post_b <= 10:
            return default
        
        enemy_acc = en_post_a / (en_post_a + en_post_b) 
        player_acc = p_post_a / (p_post_a + p_post_b)
        return max([enemy_acc - player_acc/2.0, lower_bound])
    
    def get_variance(self, base_variance, reduc_variance,  additional_variance, step,\
                     max_counter = 250, window = 20, power = 2.5, buffer = 80):
        perc_97 = {25: 16.0, 26: 13.0, 27: 11.0, 28: 7.5, 29: 6.5, 30: 5.66667, 31: 5.33333, 32: 4.66917, 33: 4.2525, 34: 4.0, 35: 3.503, 36: 3.4, 37: 3.28714, 38: 3.00429, 39: 3.0, 40: 2.83333, 41: 2.75, 42: 2.71429, 43: 2.60075, 44: 2.44444, 45: 2.4, 46: 2.36398, 47: 2.3, 48: 2.23135, 49: 2.20067, 50: 2.15423, 51: 2.10033, 52: 2.07159, 53: 2.0, 54: 2.0, 55: 2.0, 56: 1.9375, 57: 1.91686, 58: 1.88272, 59: 1.85714, 60: 1.82353, 61: 1.81267, 62: 1.78947, 63: 1.76471, 64: 1.72727, 65: 1.71429, 66: 1.68431, 67: 1.68455, 68: 1.66667, 69: 1.65007, 70: 1.64706, 71: 1.62969, 72: 1.61905, 73: 1.59281, 74: 1.58073, 75: 1.57143, 76: 1.55569, 77: 1.54559, 78: 1.54178, 79: 1.54167, 80: 1.52005, 81: 1.51852, 82: 1.5, 83: 1.5, 84: 1.48282, 85: 1.48, 86: 1.48276, 87: 1.47059, 88: 1.46875, 89: 1.46154, 90: 1.45469, 91: 1.44444, 92: 1.44118, 93: 1.43761, 94: 1.41667, 95: 1.40541, 96: 1.40016, 97: 1.4, 98: 1.39489, 99: 1.38906, 100: 1.38889, 101: 1.38715, 102: 1.38462, 103: 1.37846, 104: 1.38255, 105: 1.36851, 106: 1.37143, 107: 1.36119, 108: 1.3556, 109: 1.3556, 110: 1.35, 111: 1.34148, 112: 1.34091, 113: 1.33333, 114: 1.32673, 115: 1.32581, 116: 1.3256, 117: 1.31707, 118: 1.30775, 119: 1.30435, 120: 1.3019, 121: 1.3, 122: 1.29787, 123: 1.2917, 124: 1.2858, 125: 1.28571, 126: 1.28571, 127: 1.28579, 128: 1.28076, 129: 1.2787, 130: 1.27273, 131: 1.27454, 132: 1.27147, 133: 1.27089, 134: 1.26415, 135: 1.2667, 136: 1.26154, 137: 1.25862, 138: 1.25498, 139: 1.25455, 140: 1.25864, 141: 1.25461, 142: 1.25011, 143: 1.24561, 144: 1.24138, 145: 1.24563, 146: 1.23819, 147: 1.23529, 148: 1.23336, 149: 1.2344, 150: 1.23218, 151: 1.22955, 152: 1.2286, 153: 1.22389, 154: 1.22228, 155: 1.22222, 156: 1.22389, 157: 1.21742, 158: 1.21739, 159: 1.21255, 160: 1.21318, 161: 1.21429, 162: 1.21129, 163: 1.21053, 164: 1.20781, 165: 1.20513, 166: 1.20253, 167: 1.19737, 168: 1.19451, 169: 1.19718, 170: 1.19718, 171: 1.19481, 172: 1.19698, 173: 1.19481, 174: 1.19281, 175: 1.19232, 176: 1.18928, 177: 1.18428, 178: 1.1856, 179: 1.18671, 180: 1.18182, 181: 1.18185, 182: 1.17952, 183: 1.17857, 184: 1.17726, 185: 1.17722, 186: 1.17808, 187: 1.17568, 188: 1.17778, 189: 1.17582, 190: 1.17527, 191: 1.17393, 192: 1.17105, 193: 1.16854, 194: 1.16667, 195: 1.16842, 196: 1.16484, 197: 1.16484, 198: 1.16304, 199: 1.1596, 200: 1.15909, 201: 1.15911, 202: 1.15911, 203: 1.15844, 204: 1.1596, 205: 1.16129, 206: 1.15963, 207: 1.15789, 208: 1.15625, 209: 1.15387, 210: 1.15095, 211: 1.1524, 212: 1.14898, 213: 1.14817, 214: 1.14853, 215: 1.14567, 216: 1.14415, 217: 1.1429, 218: 1.14433, 219: 1.14588, 220: 1.14437, 221: 1.14286, 222: 1.14151, 223: 1.14004, 224: 1.13835, 225: 1.1389, 226: 1.13725, 227: 1.13675, 228: 1.13761, 229: 1.13594, 230: 1.13514, 231: 1.13403, 232: 1.13335, 233: 1.1316, 234: 1.13158, 235: 1.13159, 236: 1.13276, 237: 1.13266, 238: 1.13158, 239: 1.13116, 240: 1.12847, 241: 1.12844, 242: 1.12712, 243: 1.12727, 244: 1.12747, 245: 1.12747, 246: 1.12621, 247: 1.125, 248: 1.125, 249: 1.12503, 250: 1.125, 251: 1.12295, 252: 1.12295, 253: 1.12097, 254: 1.12039, 255: 1.12032, 256: 1.12122, 257: 1.12308, 258: 1.12195, 259: 1.12195, 260: 1.12097, 261: 1.11595, 262: 1.1157, 263: 1.1167, 264: 1.11608, 265: 1.11538, 266: 1.11539, 267: 1.11268, 268: 1.11429, 269: 1.11429, 270: 1.11348, 271: 1.11209, 272: 1.11189, 273: 1.11111, 274: 1.10961, 275: 1.11024, 276: 1.10961, 277: 1.10938, 278: 1.10951, 279: 1.1087, 280: 1.10745, 281: 1.1087, 282: 1.1087, 283: 1.10791, 284: 1.1077, 285: 1.10886, 286: 1.10813, 287: 1.10714, 288: 1.1064, 289: 1.10692, 290: 1.1064, 291: 1.10667, 292: 1.10607, 293: 1.10527, 294: 1.10373, 295: 1.10417, 296: 1.10145, 297: 1.10145, 298: 1.10296, 299: 1.10147, 300: 1.10067, 301: 1.10079, 302: 1.10004, 303: 1.10135, 304: 1.10002, 305: 1.10002, 306: 1.1, 307: 1.09878, 308: 1.09792, 309: 1.09792, 310: 1.09758, 311: 1.09725, 312: 1.09494, 313: 1.09632, 314: 1.09461, 315: 1.09376, 316: 1.09459, 317: 1.09554, 318: 1.09435, 319: 1.09555, 320: 1.09335, 321: 1.09302, 322: 1.09317, 323: 1.0926, 324: 1.09221, 325: 1.09203, 326: 1.09203, 327: 1.09146, 328: 1.09196, 329: 1.09202, 330: 1.09249, 331: 1.09196, 332: 1.09146, 333: 1.09093, 334: 1.0915, 335: 1.09093, 336: 1.09091, 337: 1.0914, 338: 1.09146, 339: 1.09036, 340: 1.0899, 341: 1.08929, 342: 1.09091, 343: 1.08784, 344: 1.08806, 345: 1.08643, 346: 1.08621, 347: 1.08557, 348: 1.08434, 349: 1.08335, 350: 1.08434, 351: 1.08427, 352: 1.0838, 353: 1.0838, 354: 1.08333, 355: 1.08235, 356: 1.08243, 357: 1.08235, 358: 1.08286, 359: 1.08333, 360: 1.08284, 361: 1.08333, 362: 1.0829, 363: 1.08335, 364: 1.0825, 365: 1.08243, 366: 1.08235, 367: 1.08198, 368: 1.0825, 369: 1.08333, 370: 1.08287, 371: 1.08291, 372: 1.08197, 373: 1.08, 374: 1.0791, 375: 1.07866, 376: 1.0791, 377: 1.07937, 378: 1.07843, 379: 1.07854, 380: 1.07692, 381: 1.07773, 382: 1.07853, 383: 1.07692, 384: 1.07732, 385: 1.07692, 386: 1.0765, 387: 1.07515, 388: 1.07548, 389: 1.07603, 390: 1.07583, 391: 1.07609, 392: 1.07576, 393: 1.07577, 394: 1.07503, 395: 1.07501, 396: 1.07515, 397: 1.07577, 398: 1.07538, 399: 1.07463, 400: 1.07426, 401: 1.07292, 402: 1.07283, 403: 1.07247, 404: 1.07255, 405: 1.07212, 406: 1.07144, 407: 1.07177, 408: 1.0711, 409: 1.07107, 410: 1.07071, 411: 1.07071, 412: 1.0701, 413: 1.07076, 414: 1.07, 415: 1.06931, 416: 1.06863, 417: 1.06863, 418: 1.0683, 419: 1.06764, 420: 1.06771, 421: 1.06764, 422: 1.06987, 423: 1.06914, 424: 1.06931, 425: 1.06757, 426: 1.06851, 427: 1.06881, 428: 1.06863, 429: 1.06863, 430: 1.06819, 431: 1.06728, 432: 1.06604, 433: 1.06575, 434: 1.06637, 435: 1.06727, 436: 1.06726, 437: 1.06558, 438: 1.06467, 439: 1.06543, 440: 1.06573, 441: 1.06523, 442: 1.06494, 443: 1.06343, 444: 1.06438, 445: 1.06452, 446: 1.06365, 447: 1.06393, 448: 1.06356, 449: 1.06365, 450: 1.06329, 451: 1.06276, 452: 1.06168, 453: 1.06168, 454: 1.0625, 455: 1.06337, 456: 1.0625, 457: 1.06222, 458: 1.0633, 459: 1.06167, 460: 1.06167, 461: 1.0614, 462: 1.06167, 463: 1.06168, 464: 1.0622, 465: 1.06277, 466: 1.06174, 467: 1.06173, 468: 1.06122, 469: 1.06141, 470: 1.06061, 471: 1.06098, 472: 1.06036, 473: 1.05983, 474: 1.05983, 475: 1.0591, 476: 1.05977, 477: 1.05856, 478: 1.05856, 479: 1.05834, 480: 1.05769, 481: 1.0577, 482: 1.05738, 483: 1.05761, 484: 1.05669, 485: 1.05761, 486: 1.05682, 487: 1.05692, 488: 1.05691, 489: 1.05668, 490: 1.05534, 491: 1.05581, 492: 1.05579, 493: 1.05622, 494: 1.056, 495: 1.05578, 496: 1.05601, 497: 1.0558, 498: 1.05534, 499: 1.05534, 500: 1.05512, 501: 1.05578, 502: 1.05535, 503: 1.05515, 504: 1.05536, 505: 1.05556, 506: 1.05495, 507: 1.05426, 508: 1.0549, 509: 1.05462, 510: 1.05515, 511: 1.05448, 512: 1.05455, 513: 1.05447, 514: 1.05364, 515: 1.05307, 516: 1.05328, 517: 1.05324, 518: 1.05284, 519: 1.05243, 520: 1.05209, 521: 1.05159, 522: 1.05167, 523: 1.05204, 524: 1.05085, 525: 1.0508, 526: 1.05036, 527: 1.05018, 528: 1.05019, 529: 1.04983, 530: 1.04982, 531: 1.04981, 532: 1.05001, 533: 1.05001, 534: 1.05, 535: 1.04981, 536: 1.04981, 537: 1.04925, 538: 1.05001, 539: 1.05036, 540: 1.04898, 541: 1.04906, 542: 1.04878, 543: 1.04924, 544: 1.04906, 545: 1.048, 546: 1.048, 547: 1.048, 548: 1.04715, 549: 1.04762, 550: 1.0473, 551: 1.04815, 552: 1.04743, 553: 1.04688, 554: 1.04643, 555: 1.04688, 556: 1.04698, 557: 1.04651, 558: 1.04636, 559: 1.04545, 560: 1.04478, 561: 1.04514, 562: 1.04483, 563: 1.04517, 564: 1.04545, 565: 1.04605, 566: 1.04546, 567: 1.0458, 568: 1.04617, 569: 1.04563, 570: 1.04514, 571: 1.0453, 572: 1.04478, 573: 1.04483, 574: 1.04461, 575: 1.04514, 576: 1.0453, 577: 1.0453, 578: 1.04478, 579: 1.04494, 580: 1.04546, 581: 1.04479, 582: 1.04468, 583: 1.04462, 584: 1.0438, 585: 1.0438, 586: 1.04363, 587: 1.04392, 588: 1.04334, 589: 1.04396, 590: 1.04445, 591: 1.04333, 592: 1.0438, 593: 1.04332, 594: 1.04306, 595: 1.04392, 596: 1.0428, 597: 1.04211, 598: 1.0423, 599: 1.04235, 600: 1.04138, 601: 1.0414, 602: 1.04101, 603: 1.04096, 604: 1.04127, 605: 1.04013, 606: 1.04027, 607: 1.0403, 608: 1.04127, 609: 1.04168, 610: 1.04154, 611: 1.0411, 612: 1.04088, 613: 1.04207, 614: 1.04088, 615: 1.04136, 616: 1.04137, 617: 1.04083, 618: 1.04054, 619: 1.03976, 620: 1.04027, 621: 1.03955, 622: 1.0403, 623: 1.03962, 624: 1.03947, 625: 1.03943, 626: 1.0405, 627: 1.04124, 628: 1.04, 629: 1.03989, 630: 1.03989, 631: 1.03987, 632: 1.04014, 633: 1.04088, 634: 1.04028, 635: 1.03987, 636: 1.04115, 637: 1.04127, 638: 1.04096, 639: 1.04044, 640: 1.03964, 641: 1.04075, 642: 1.03948, 643: 1.03935, 644: 1.03935, 645: 1.03881, 646: 1.03824, 647: 1.03791, 648: 1.03797, 649: 1.03767, 650: 1.03691, 651: 1.03681, 652: 1.03704, 653: 1.0367, 654: 1.03642, 655: 1.03655, 656: 1.03654, 657: 1.03643, 658: 1.03624, 659: 1.03717, 660: 1.03738, 661: 1.03725, 662: 1.03704, 663: 1.03692, 664: 1.03637, 665: 1.0365, 666: 1.03682, 667: 1.03648, 668: 1.03659, 669: 1.03643, 670: 1.03647, 671: 1.03647, 672: 1.0367, 673: 1.03624, 674: 1.0367, 675: 1.03637, 676: 1.03625, 677: 1.03538, 678: 1.03572, 679: 1.03509, 680: 1.03604, 681: 1.03459, 682: 1.035, 683: 1.03469, 684: 1.03469, 685: 1.03571, 686: 1.03483, 687: 1.035, 688: 1.03468, 689: 1.03509, 690: 1.03528, 691: 1.03548, 692: 1.03593, 693: 1.03582, 694: 1.03534, 695: 1.03481, 696: 1.03479, 697: 1.03436, 698: 1.03499, 699: 1.03571, 700: 1.03615, 701: 1.03515, 702: 1.03529, 703: 1.0353, 704: 1.03515, 705: 1.03509, 706: 1.03509, 707: 1.03499, 708: 1.03458, 709: 1.03449, 710: 1.0345, 711: 1.03489, 712: 1.03493, 713: 1.03503, 714: 1.03469, 715: 1.03439, 716: 1.034, 717: 1.034, 718: 1.03381, 719: 1.03344, 720: 1.03356, 721: 1.03362, 722: 1.03345, 723: 1.03334, 724: 1.03314, 725: 1.03334, 726: 1.03313, 727: 1.03323, 728: 1.03333, 729: 1.03293, 730: 1.0327, 731: 1.03284, 732: 1.03304, 733: 1.03274, 734: 1.03208, 735: 1.03161, 736: 1.03179, 737: 1.03115, 738: 1.03161, 739: 1.03198, 740: 1.03226, 741: 1.03217, 742: 1.03217, 743: 1.03226, 744: 1.0318, 745: 1.03189, 746: 1.03198, 747: 1.03198, 748: 1.03198, 749: 1.03216, 750: 1.03189, 751: 1.0319, 752: 1.03226, 753: 1.03195, 754: 1.03158, 755: 1.03161, 756: 1.0315, 757: 1.03142, 758: 1.03183, 759: 1.03152, 760: 1.03144, 761: 1.03049, 762: 1.0307, 763: 1.03049, 764: 1.03062, 765: 1.03021, 766: 1.02997, 767: 1.03039, 768: 1.02961, 769: 1.02959, 770: 1.03003, 771: 1.0303, 772: 1.02994, 773: 1.03006, 774: 1.03039, 775: 1.02981, 776: 1.03003, 777: 1.02959, 778: 1.02903, 779: 1.02982, 780: 1.02994, 781: 1.02942, 782: 1.02985, 783: 1.02973, 784: 1.03021, 785: 1.03021, 786: 1.03013, 787: 1.03013, 788: 1.02998, 789: 1.03065, 790: 1.03099, 791: 1.0309, 792: 1.03073, 793: 1.03062, 794: 1.03014, 795: 1.02934, 796: 1.02881, 797: 1.02918, 798: 1.02882, 799: 1.02865, 800: 1.02857, 801: 1.02907, 802: 1.02891, 803: 1.02926, 804: 1.02818, 805: 1.02843, 806: 1.02874, 807: 1.02873, 808: 1.02864, 809: 1.02829, 810: 1.02785, 811: 1.02813, 812: 1.02829, 813: 1.02866, 814: 1.02841, 815: 1.02837, 816: 1.02842, 817: 1.02786, 818: 1.02737, 819: 1.02717, 820: 1.02801, 821: 1.02792, 822: 1.02757, 823: 1.02787, 824: 1.02763, 825: 1.02799, 826: 1.02836, 827: 1.02873, 828: 1.02849, 829: 1.02913, 830: 1.02857, 831: 1.02857, 832: 1.02825, 833: 1.02843, 834: 1.02801, 835: 1.02833, 836: 1.0285, 837: 1.02888, 838: 1.02817, 839: 1.02866, 840: 1.02858, 841: 1.02785, 842: 1.02778, 843: 1.02769, 844: 1.02703, 845: 1.02674, 846: 1.02667, 847: 1.02639, 848: 1.0271, 849: 1.02687, 850: 1.02801, 851: 1.02712, 852: 1.02733, 853: 1.02757, 854: 1.02748, 855: 1.02676, 856: 1.02792, 857: 1.0271, 858: 1.02677, 859: 1.0266, 860: 1.02639, 861: 1.0271, 862: 1.02653, 863: 1.0271, 864: 1.02718, 865: 1.0269, 866: 1.02725, 867: 1.02683, 868: 1.02683, 869: 1.02684, 870: 1.0266, 871: 1.02632, 872: 1.02677, 873: 1.02638, 874: 1.02697, 875: 1.02646, 876: 1.02646, 877: 1.02674, 878: 1.02674, 879: 1.02709, 880: 1.02674, 881: 1.02712, 882: 1.02717, 883: 1.02619, 884: 1.02669, 885: 1.02709, 886: 1.02684, 887: 1.02646, 888: 1.02624, 889: 1.02625, 890: 1.02616, 891: 1.02611, 892: 1.02632, 893: 1.02626, 894: 1.02598, 895: 1.02646, 896: 1.02626, 897: 1.02618, 898: 1.02639, 899: 1.02558, 900: 1.02532, 901: 1.02594, 902: 1.02558, 903: 1.02571, 904: 1.02576, 905: 1.02576, 906: 1.02551, 907: 1.025, 908: 1.02512, 909: 1.02486, 910: 1.02482, 911: 1.02494, 912: 1.02476, 913: 1.02469, 914: 1.02532, 915: 1.02507, 916: 1.02507, 917: 1.02539, 918: 1.02552, 919: 1.02529, 920: 1.02564, 921: 1.02589, 922: 1.02519, 923: 1.02529, 924: 1.02506, 925: 1.02523, 926: 1.02466, 927: 1.0246, 928: 1.02475, 929: 1.02467, 930: 1.02494, 931: 1.02475, 932: 1.02444, 933: 1.02457, 934: 1.02478, 935: 1.02481, 936: 1.02457, 937: 1.02488, 938: 1.02488, 939: 1.02464, 940: 1.02545, 941: 1.02507, 942: 1.02545, 943: 1.02483, 944: 1.02545, 945: 1.02545, 946: 1.02507, 947: 1.02501, 948: 1.02433, 949: 1.02451, 950: 1.02513, 951: 1.02525, 952: 1.02456, 953: 1.02541, 954: 1.02519, 955: 1.02546, 956: 1.02547, 957: 1.025, 958: 1.02507, 959: 1.02541, 960: 1.02439, 961: 1.02469, 962: 1.02501, 963: 1.02546, 964: 1.02564, 965: 1.02541, 966: 1.02513, 967: 1.02427, 968: 1.02413, 969: 1.02348, 970: 1.02413, 971: 1.02404, 972: 1.02337, 973: 1.02336, 974: 1.02331, 975: 1.02344, 976: 1.02337, 977: 1.02404, 978: 1.02353, 979: 1.02359, 980: 1.02359, 981: 1.02376, 982: 1.02353, 983: 1.0232, 984: 1.02338, 985: 1.02362, 986: 1.02377, 987: 1.02353, 988: 1.02321, 989: 1.02308, 990: 1.02376, 991: 1.02375, 992: 1.0237, 993: 1.02304, 994: 1.02279, 995: 1.0232, 996: 1.02294, 997: 1.02326, 998: 1.02366, 999: 1.0237, 1000: 1.02371, 1001: 1.02349, 1002: 1.02343, 1003: 1.02321, 1004: 1.02279, 1005: 1.02331, 1006: 1.02348, 1007: 1.02273, 1008: 1.0237, 1009: 1.02326, 1010: 1.02315, 1011: 1.02326, 1012: 1.02388, 1013: 1.02353, 1014: 1.02331, 1015: 1.02353, 1016: 1.02321, 1017: 1.02273, 1018: 1.02283, 1019: 1.02273, 1020: 1.02228, 1021: 1.02208, 1022: 1.02243, 1023: 1.02238, 1024: 1.02181, 1025: 1.02158, 1026: 1.02238, 1027: 1.0231, 1028: 1.02299, 1029: 1.02262, 1030: 1.02228, 1031: 1.02263, 1032: 1.0225, 1033: 1.02279, 1034: 1.02267, 1035: 1.02273, 1036: 1.02273, 1037: 1.02267, 1038: 1.02315, 1039: 1.02294, 1040: 1.02304, 1041: 1.02304, 1042: 1.02337, 1043: 1.02311, 1044: 1.02285, 1045: 1.02263, 1046: 1.02262, 1047: 1.02242, 1048: 1.02248, 1049: 1.02247, 1050: 1.02217, 1051: 1.02201, 1052: 1.02218, 1053: 1.02228, 1054: 1.02213, 1055: 1.02228, 1056: 1.02213, 1057: 1.02188, 1058: 1.02188, 1059: 1.02188, 1060: 1.02185, 1061: 1.02148, 1062: 1.02128, 1063: 1.02113, 1064: 1.0211, 1065: 1.0211, 1066: 1.02143, 1067: 1.02174, 1068: 1.02158, 1069: 1.02184, 1070: 1.02174, 1071: 1.02119, 1072: 1.02179, 1073: 1.02153, 1074: 1.02133, 1075: 1.02128, 1076: 1.02148, 1077: 1.02148, 1078: 1.02188, 1079: 1.02108, 1080: 1.02093, 1081: 1.02103, 1082: 1.02108, 1083: 1.02119, 1084: 1.02098, 1085: 1.02036, 1086: 1.02102, 1087: 1.02119, 1088: 1.02083, 1089: 1.02018, 1090: 1.02036, 1091: 1.02069, 1092: 1.02075, 1093: 1.02051, 1094: 1.02054, 1095: 1.02074, 1096: 1.02088, 1097: 1.0207, 1098: 1.02055, 1099: 1.02055, 1100: 1.02055, 1101: 1.02005, 1102: 1.01985, 1103: 1.01996, 1104: 1.01996, 1105: 1.02024, 1106: 1.02037, 1107: 1.02041, 1108: 1.02005, 1109: 1.02008, 1110: 1.0194, 1111: 1.01923, 1112: 1.01961, 1113: 1.01983, 1114: 1.01947, 1115: 1.01969, 1116: 1.01978, 1117: 1.02025, 1118: 1.02037, 1119: 1.02023, 1120: 1.02014, 1121: 1.01965, 1122: 1.02016, 1123: 1.02005, 1124: 1.01983, 1125: 1.02, 1126: 1.01973, 1127: 1.01948, 1128: 1.01936, 1129: 1.0194, 1130: 1.0194, 1131: 1.01966, 1132: 1.01987, 1133: 1.01987, 1134: 1.01952, 1135: 1.01953, 1136: 1.0194, 1137: 1.01904, 1138: 1.0194, 1139: 1.01961, 1140: 1.0197, 1141: 1.01988, 1142: 1.01891, 1143: 1.01937, 1144: 1.01974, 1145: 1.01969, 1146: 1.01969, 1147: 1.0194, 1148: 1.01989, 1149: 1.02, 1150: 1.02029, 1151: 1.02028, 1152: 1.01969, 1153: 1.01969, 1154: 1.01928, 1155: 1.0198, 1156: 1.02, 1157: 1.01919, 1158: 1.01957, 1159: 1.01907, 1160: 1.01915, 1161: 1.01915, 1162: 1.0194, 1163: 1.01856, 1164: 1.01879, 1165: 1.01896, 1166: 1.01923, 1167: 1.01875, 1168: 1.01899, 1169: 1.0187, 1170: 1.01944, 1171: 1.01944, 1172: 1.01841, 1173: 1.01844, 1174: 1.01833, 1175: 1.01856, 1176: 1.01927, 1177: 1.01884, 1178: 1.01879, 1179: 1.01871, 1180: 1.01871, 1181: 1.01856, 1182: 1.01826, 1183: 1.0183, 1184: 1.01852, 1185: 1.01822, 1186: 1.01848, 1187: 1.01826, 1188: 1.01848, 1189: 1.01782, 1190: 1.01811, 1191: 1.01786, 1192: 1.01811, 1193: 1.01794, 1194: 1.0179, 1195: 1.01743, 1196: 1.01761, 1197: 1.01786, 1198: 1.0181, 1199: 1.01827, 1200: 1.01786, 1201: 1.01806, 1202: 1.01786, 1203: 1.01802, 1204: 1.01786, 1205: 1.01779, 1206: 1.01808, 1207: 1.01775, 1208: 1.01793, 1209: 1.01794, 1210: 1.01794, 1211: 1.01772, 1212: 1.01782, 1213: 1.01759, 1214: 1.01748, 1215: 1.01758, 1216: 1.01736, 1217: 1.01747, 1218: 1.01736, 1219: 1.01751, 1220: 1.01778, 1221: 1.01772, 1222: 1.0179, 1223: 1.018, 1224: 1.01808, 1225: 1.01748, 1226: 1.0177, 1227: 1.01755, 1228: 1.01744, 1229: 1.01724, 1230: 1.01711, 1231: 1.01708, 1232: 1.01718, 1233: 1.01681, 1234: 1.01758, 1235: 1.01775, 1236: 1.01786, 1237: 1.01729, 1238: 1.01758, 1239: 1.01734, 1240: 1.01725, 1241: 1.01715, 1242: 1.01689, 1243: 1.01728, 1244: 1.01728, 1245: 1.01728, 1246: 1.01661, 1247: 1.01677, 1248: 1.01713, 1249: 1.01721, 1250: 1.01717, 1251: 1.01695, 1252: 1.01713, 1253: 1.01714, 1254: 1.01702, 1255: 1.01721, 1256: 1.01735, 1257: 1.01717, 1258: 1.01734, 1259: 1.01744, 1260: 1.01744, 1261: 1.01759, 1262: 1.01702, 1263: 1.01744, 1264: 1.01751, 1265: 1.01751, 1266: 1.01728, 1267: 1.01728, 1268: 1.01728, 1269: 1.01748, 1270: 1.01747, 1271: 1.01724, 1272: 1.01754, 1273: 1.01786, 1274: 1.01709, 1275: 1.01699, 1276: 1.01702, 1277: 1.01679, 1278: 1.01686, 1279: 1.01705, 1280: 1.01692, 1281: 1.01679, 1282: 1.01688, 1283: 1.01667, 1284: 1.01667, 1285: 1.01663, 1286: 1.01674, 1287: 1.01681, 1288: 1.01667, 1289: 1.01667, 1290: 1.01658, 1291: 1.01649, 1292: 1.01643, 1293: 1.01681, 1294: 1.01711, 1295: 1.01702, 1296: 1.01717, 1297: 1.01717, 1298: 1.01674, 1299: 1.01671, 1300: 1.01633, 1301: 1.01617, 1302: 1.0163, 1303: 1.01623, 1304: 1.01613, 1305: 1.01661, 1306: 1.01637, 1307: 1.0162, 1308: 1.0162, 1309: 1.01616, 1310: 1.01616, 1311: 1.01613, 1312: 1.01626, 1313: 1.01579, 1314: 1.01575, 1315: 1.01554, 1316: 1.01487, 1317: 1.0149, 1318: 1.01539, 1319: 1.01499, 1320: 1.01606, 1321: 1.016, 1322: 1.01616, 1323: 1.01585, 1324: 1.01601, 1325: 1.01606, 1326: 1.01549, 1327: 1.01539, 1328: 1.01538, 1329: 1.01542, 1330: 1.01554, 1331: 1.01549, 1332: 1.01581, 1333: 1.01581, 1334: 1.0156, 1335: 1.01557, 1336: 1.01553, 1337: 1.01542, 1338: 1.01554, 1339: 1.0155, 1340: 1.01551, 1341: 1.01533, 1342: 1.01518, 1343: 1.01496, 1344: 1.0151, 1345: 1.01533, 1346: 1.01557, 1347: 1.01527, 1348: 1.01566, 1349: 1.01549, 1350: 1.01557, 1351: 1.01498, 1352: 1.0149, 1353: 1.01471, 1354: 1.01471, 1355: 1.01487, 1356: 1.01528, 1357: 1.01528, 1358: 1.01493, 1359: 1.01563, 1360: 1.01486, 1361: 1.0149, 1362: 1.01524, 1363: 1.01489, 1364: 1.01512, 1365: 1.01557, 1366: 1.01569, 1367: 1.0151, 1368: 1.0148, 1369: 1.01487, 1370: 1.01486, 1371: 1.01495, 1372: 1.01523, 1373: 1.01501, 1374: 1.01476, 1375: 1.01479, 1376: 1.01533, 1377: 1.0155, 1378: 1.01553, 1379: 1.01533, 1380: 1.01502, 1381: 1.01512, 1382: 1.01527, 1383: 1.01527, 1384: 1.01536, 1385: 1.01493, 1386: 1.01495, 1387: 1.01495, 1388: 1.01495, 1389: 1.01524, 1390: 1.0151, 1391: 1.01539, 1392: 1.01533, 1393: 1.01544, 1394: 1.01539, 1395: 1.01539, 1396: 1.01552, 1397: 1.01594, 1398: 1.01545, 1399: 1.01545, 1400: 1.01527, 1401: 1.01518, 1402: 1.01524, 1403: 1.01524, 1404: 1.01509, 1405: 1.01495, 1406: 1.0151, 1407: 1.0153, 1408: 1.01572, 1409: 1.01536, 1410: 1.01507, 1411: 1.01479, 1412: 1.01479, 1413: 1.01468, 1414: 1.01468, 1415: 1.01455, 1416: 1.01465, 1417: 1.01465, 1418: 1.01463, 1419: 1.01484, 1420: 1.01518, 1421: 1.01477, 1422: 1.01462, 1423: 1.01479, 1424: 1.01493, 1425: 1.01493, 1426: 1.0145, 1427: 1.01447, 1428: 1.01471, 1429: 1.0148, 1430: 1.0148, 1431: 1.0148, 1432: 1.01468, 1433: 1.01477, 1434: 1.01482, 1435: 1.01443, 1436: 1.01471, 1437: 1.01487, 1438: 1.01477, 1439: 1.01487, 1440: 1.01457, 1441: 1.01498, 1442: 1.01465, 1443: 1.01463, 1444: 1.01465, 1445: 1.01457, 1446: 1.01477, 1447: 1.0146, 1448: 1.01468, 1449: 1.01452, 1450: 1.01452, 1451: 1.01447, 1452: 1.01452, 1453: 1.01452, 1454: 1.01458, 1455: 1.01458, 1456: 1.01429, 1457: 1.01442, 1458: 1.01431, 1459: 1.01452, 1460: 1.01465, 1461: 1.01452, 1462: 1.01421, 1463: 1.01424, 1464: 1.01397, 1465: 1.01402, 1466: 1.01401, 1467: 1.01411, 1468: 1.01413, 1469: 1.01444, 1470: 1.01447, 1471: 1.01409, 1472: 1.01409, 1473: 1.01403, 1474: 1.01396, 1475: 1.01403, 1476: 1.01396, 1477: 1.01407, 1478: 1.01431, 1479: 1.01404, 1480: 1.01401, 1481: 1.01429, 1482: 1.01403, 1483: 1.01384, 1484: 1.01373, 1485: 1.01384, 1486: 1.01365, 1487: 1.01379, 1488: 1.01331, 1489: 1.01354, 1490: 1.01341, 1491: 1.01372, 1492: 1.01362, 1493: 1.01386, 1494: 1.01368, 1495: 1.01362, 1496: 1.01365, 1497: 1.01352, 1498: 1.01365, 1499: 1.0134, 1500: 1.01341, 1501: 1.01334, 1502: 1.01346, 1503: 1.01339, 1504: 1.01339, 1505: 1.01338, 1506: 1.01356, 1507: 1.01356, 1508: 1.01358, 1509: 1.01334, 1510: 1.01321, 1511: 1.01318, 1512: 1.0132, 1513: 1.01352, 1514: 1.01358, 1515: 1.0137, 1516: 1.01368, 1517: 1.01359, 1518: 1.01299, 1519: 1.01356, 1520: 1.01373, 1521: 1.01363, 1522: 1.01357, 1523: 1.01355, 1524: 1.01382, 1525: 1.01378, 1526: 1.01354, 1527: 1.01375, 1528: 1.01379, 1529: 1.01392, 1530: 1.01379, 1531: 1.01351, 1532: 1.01326, 1533: 1.01297, 1534: 1.01304, 1535: 1.01297, 1536: 1.01314, 1537: 1.01314, 1538: 1.01308, 1539: 1.01308, 1540: 1.01311, 1541: 1.01309, 1542: 1.01301, 1543: 1.01318, 1544: 1.01306, 1545: 1.01294, 1546: 1.01294, 1547: 1.01282, 1548: 1.01268, 1549: 1.01266, 1550: 1.01266, 1551: 1.01286, 1552: 1.01289, 1553: 1.01289, 1554: 1.01268, 1555: 1.01321, 1556: 1.01385, 1557: 1.01363, 1558: 1.01316, 1559: 1.01325, 1560: 1.01382, 1561: 1.01331, 1562: 1.01304, 1563: 1.01275, 1564: 1.01275, 1565: 1.01287, 1566: 1.01289, 1567: 1.01289, 1568: 1.01254, 1569: 1.01259, 1570: 1.0128, 1571: 1.01299, 1572: 1.01301, 1573: 1.01349, 1574: 1.01326, 1575: 1.01329, 1576: 1.01284, 1577: 1.01301, 1578: 1.0128, 1579: 1.01299, 1580: 1.01277, 1581: 1.01277, 1582: 1.01276, 1583: 1.01264, 1584: 1.0128, 1585: 1.01274, 1586: 1.01274, 1587: 1.01274, 1588: 1.01292, 1589: 1.01273, 1590: 1.01273, 1591: 1.0127, 1592: 1.0127, 1593: 1.01266, 1594: 1.01271, 1595: 1.01257, 1596: 1.01339, 1597: 1.01323, 1598: 1.01321, 1599: 1.01286, 1600: 1.01286, 1601: 1.01272, 1602: 1.01272, 1603: 1.01271, 1604: 1.01248, 1605: 1.01231, 1606: 1.01237, 1607: 1.01235, 1608: 1.0123, 1609: 1.01235, 1610: 1.01239, 1611: 1.01259, 1612: 1.01271, 1613: 1.01259, 1614: 1.01261, 1615: 1.01264, 1616: 1.0132, 1617: 1.01323, 1618: 1.01311, 1619: 1.01289, 1620: 1.01289, 1621: 1.01257, 1622: 1.01253, 1623: 1.01252, 1624: 1.01241, 1625: 1.0125, 1626: 1.01248, 1627: 1.01245, 1628: 1.01217, 1629: 1.01226, 1630: 1.01237, 1631: 1.01241, 1632: 1.01215, 1633: 1.01233, 1634: 1.01215, 1635: 1.01211, 1636: 1.01213, 1637: 1.01218, 1638: 1.01213, 1639: 1.01197, 1640: 1.01165, 1641: 1.01211, 1642: 1.01237, 1643: 1.01239, 1644: 1.01239, 1645: 1.01237, 1646: 1.01217, 1647: 1.01244, 1648: 1.01239, 1649: 1.01255, 1650: 1.01241, 1651: 1.01248, 1652: 1.01255, 1653: 1.01227, 1654: 1.01216, 1655: 1.01197, 1656: 1.01197, 1657: 1.01237, 1658: 1.0122, 1659: 1.01197, 1660: 1.01201, 1661: 1.01241, 1662: 1.01218, 1663: 1.01222, 1664: 1.01239, 1665: 1.01228, 1666: 1.01225, 1667: 1.01207, 1668: 1.01214, 1669: 1.01195, 1670: 1.01173, 1671: 1.01173, 1672: 1.01148, 1673: 1.01203, 1674: 1.01224, 1675: 1.01222, 1676: 1.0122, 1677: 1.01197, 1678: 1.01184, 1679: 1.01189, 1680: 1.01197, 1681: 1.01191, 1682: 1.01181, 1683: 1.01207, 1684: 1.01191, 1685: 1.01157, 1686: 1.01146, 1687: 1.01128, 1688: 1.01127, 1689: 1.01143, 1690: 1.01153, 1691: 1.01124, 1692: 1.01128, 1693: 1.01128, 1694: 1.0114, 1695: 1.01184, 1696: 1.01159, 1697: 1.01152, 1698: 1.01149, 1699: 1.0115, 1700: 1.01119, 1701: 1.01144, 1702: 1.01144, 1703: 1.01165, 1704: 1.01163, 1705: 1.01122, 1706: 1.01116, 1707: 1.01171, 1708: 1.01133, 1709: 1.0115, 1710: 1.01169, 1711: 1.0111, 1712: 1.01111, 1713: 1.01097, 1714: 1.01159, 1715: 1.01129, 1716: 1.01107, 1717: 1.01131, 1718: 1.01129, 1719: 1.01129, 1720: 1.01129, 1721: 1.01139, 1722: 1.01115, 1723: 1.01131, 1724: 1.01115, 1725: 1.01117, 1726: 1.01156, 1727: 1.0115, 1728: 1.01137, 1729: 1.01175, 1730: 1.01133, 1731: 1.01124, 1732: 1.01097, 1733: 1.01129, 1734: 1.01133, 1735: 1.01151, 1736: 1.01146, 1737: 1.01132, 1738: 1.01115, 1739: 1.01115, 1740: 1.01131, 1741: 1.01152, 1742: 1.01127, 1743: 1.01127, 1744: 1.01146, 1745: 1.01149, 1746: 1.01143, 1747: 1.01157, 1748: 1.01148, 1749: 1.01153, 1750: 1.01153, 1751: 1.01142, 1752: 1.01111, 1753: 1.01125, 1754: 1.0115, 1755: 1.01149, 1756: 1.0114, 1757: 1.0112, 1758: 1.01124, 1759: 1.0112, 1760: 1.01105, 1761: 1.01161, 1762: 1.01155, 1763: 1.01165, 1764: 1.01155, 1765: 1.01169, 1766: 1.01178, 1767: 1.01155, 1768: 1.01148, 1769: 1.01148, 1770: 1.0114, 1771: 1.01167, 1772: 1.01151, 1773: 1.01145, 1774: 1.01151, 1775: 1.01155, 1776: 1.0115, 1777: 1.01128, 1778: 1.01165, 1779: 1.01147, 1780: 1.0115, 1781: 1.0112, 1782: 1.01106, 1783: 1.01106, 1784: 1.01101, 1785: 1.01115, 1786: 1.01131, 1787: 1.01126, 1788: 1.01122, 1789: 1.01122, 1790: 1.01135, 1791: 1.01093, 1792: 1.01093, 1793: 1.01113, 1794: 1.01111, 1795: 1.01082, 1796: 1.01082, 1797: 1.01089, 1798: 1.01071, 1799: 1.0107, 1800: 1.01069, 1801: 1.01067, 1802: 1.0106, 1803: 1.0106, 1804: 1.01058, 1805: 1.01058, 1806: 1.01042, 1807: 1.01036, 1808: 1.01042, 1809: 1.01015, 1810: 1.01028, 1811: 1.01036, 1812: 1.01063, 1813: 1.01049, 1814: 1.01057, 1815: 1.01025, 1816: 1.01057, 1817: 1.0107, 1818: 1.0107, 1819: 1.01023, 1820: 1.0104, 1821: 1.01068, 1822: 1.01075, 1823: 1.01065, 1824: 1.01038, 1825: 1.01076, 1826: 1.01064, 1827: 1.01064, 1828: 1.01073, 1829: 1.01089, 1830: 1.01117, 1831: 1.01153, 1832: 1.01156, 1833: 1.01153, 1834: 1.01151, 1835: 1.01161, 1836: 1.01111, 1837: 1.01087, 1838: 1.01122, 1839: 1.01096, 1840: 1.01082, 1841: 1.01085, 1842: 1.01079, 1843: 1.01066, 1844: 1.01066, 1845: 1.01076, 1846: 1.01087, 1847: 1.01103, 1848: 1.01066, 1849: 1.0107, 1850: 1.01048, 1851: 1.01026, 1852: 1.01029, 1853: 1.0102, 1854: 1.01017, 1855: 1.01029, 1856: 1.01056, 1857: 1.01029, 1858: 1.01029, 1859: 1.00992, 1860: 1.00997, 1861: 1.01007, 1862: 1.01007, 1863: 1.0099, 1864: 1.00997, 1865: 1.01002, 1866: 1.00998, 1867: 1.00987, 1868: 1.00976, 1869: 1.01005, 1870: 1.01014, 1871: 1.00995, 1872: 1.00995, 1873: 1.01003, 1874: 1.00998, 1875: 1.00982, 1876: 1.01005, 1877: 1.01007, 1878: 1.00994, 1879: 1.00995, 1880: 1.00997, 1881: 1.00997, 1882: 1.00993, 1883: 1.00997, 1884: 1.0101, 1885: 1.0099, 1886: 1.01009, 1887: 1.01029, 1888: 1.01006, 1889: 1.01006, 1890: 1.0101, 1891: 1.0104, 1892: 1.01026, 1893: 1.01027, 1894: 1.00982, 1895: 1.00993, 1896: 1.01009, 1897: 1.01019, 1898: 1.01036, 1899: 1.01036, 1900: 1.0102, 1901: 1.01022, 1902: 1.01036, 1903: 1.01037, 1904: 1.00993, 1905: 1.01006, 1906: 1.01036, 1907: 1.01049, 1908: 1.01055, 1909: 1.01037, 1910: 1.01, 1911: 1.01, 1912: 1.01012, 1913: 1.00985, 1914: 1.00987, 1915: 1.00987, 1916: 1.00992, 1917: 1.01005, 1918: 1.0099, 1919: 1.00971, 1920: 1.00999, 1921: 1.00994, 1922: 1.00962, 1923: 1.0098, 1924: 1.00994, 1925: 1.01019, 1926: 1.01019, 1927: 1.01019, 1928: 1.01015, 1929: 1.01026, 1930: 1.01015, 1931: 1.00999, 1932: 1.01015, 1933: 1.01007, 1934: 1.01017, 1935: 1.01, 1936: 1.0098, 1937: 1.00973, 1938: 1.00965, 1939: 1.00969, 1940: 1.00993, 1941: 1.00963, 1942: 1.00976, 1943: 1.00976, 1944: 1.00976, 1945: 1.00987, 1946: 1.00989, 1947: 1.00973, 1948: 1.00984, 1949: 1.00951, 1950: 1.00928, 1951: 1.00948, 1952: 1.00959, 1953: 1.00937, 1954: 1.00962, 1955: 1.00937, 1956: 1.00937, 1957: 1.00937, 1958: 1.00945, 1959: 1.00955, 1960: 1.00945, 1961: 1.00938, 1962: 1.00936, 1963: 1.00955, 1964: 1.00965, 1965: 1.00962, 1966: 1.00962, 1967: 1.00957, 1968: 1.00952, 1969: 1.00949, 1970: 1.00976, 1971: 1.0095, 1972: 1.00944, 1973: 1.00948, 1974: 1.00948, 1975: 1.00948, 1976: 1.00982, 1977: 1.0096, 1978: 1.00945, 1979: 1.00959, 1980: 1.00959, 1981: 1.00948, 1982: 1.00957, 1983: 1.00957, 1984: 1.00971, 1985: 1.00994, 1986: 1.00951, 1987: 1.00966, 1988: 1.00966, 1989: 1.00954, 1990: 1.00949, 1991: 1.00946, 1992: 1.00946, 1993: 1.00939, 1994: 1.00935, 1995: 1.00948, 1996: 1.00945, 1997: 1.00945, 1998: 1.00945, 1999: 1.00936}
        perc_50 ={21: 17.0, 22: 12.0, 23: 8.0, 24: 6.0, 25: 5.0, 26: 4.25, 27: 3.75, 28: 3.5, 29: 3.2, 30: 3.0, 31: 2.8, 32: 2.66667, 33: 2.5, 34: 2.4, 35: 2.33333, 36: 2.25, 37: 2.16026, 38: 2.11111, 39: 2.08333, 40: 2.0, 41: 1.92308, 42: 1.90909, 43: 1.85714, 44: 1.83333, 45: 1.78571, 46: 1.76471, 47: 1.75, 48: 1.71429, 49: 1.69231, 50: 1.66667, 51: 1.64853, 52: 1.63158, 53: 1.61538, 54: 1.6, 55: 1.58824, 56: 1.57143, 57: 1.55556, 58: 1.53846, 59: 1.52381, 60: 1.52087, 61: 1.5, 62: 1.48, 63: 1.47619, 64: 1.46154, 65: 1.45644, 66: 1.44444, 67: 1.43478, 68: 1.42308, 69: 1.41935, 70: 1.40825, 71: 1.4, 72: 1.39286, 73: 1.38462, 74: 1.375, 75: 1.36626, 76: 1.35714, 77: 1.35294, 78: 1.34483, 79: 1.34211, 80: 1.33333, 81: 1.32393, 82: 1.32353, 83: 1.32353, 84: 1.31579, 85: 1.30769, 86: 1.30556, 87: 1.29758, 88: 1.2973, 89: 1.2873, 90: 1.28233, 91: 1.28205, 92: 1.27778, 93: 1.2766, 94: 1.27027, 95: 1.27027, 96: 1.27055, 97: 1.26667, 98: 1.2619, 99: 1.25714, 100: 1.25581, 101: 1.25532, 102: 1.25, 103: 1.24509, 104: 1.24, 105: 1.2381, 106: 1.2366, 107: 1.23256, 108: 1.23077, 109: 1.22642, 110: 1.22449, 111: 1.22, 112: 1.21779, 113: 1.21569, 114: 1.21353, 115: 1.2093, 116: 1.20794, 117: 1.20455, 118: 1.20572, 119: 1.2037, 120: 1.2, 121: 1.2, 122: 1.19672, 123: 1.19643, 124: 1.1946, 125: 1.19298, 126: 1.18966, 127: 1.1875, 128: 1.1875, 129: 1.1849, 130: 1.18333, 131: 1.18333, 132: 1.18182, 133: 1.18182, 134: 1.18033, 135: 1.17884, 136: 1.17742, 137: 1.17544, 138: 1.1746, 139: 1.17143, 140: 1.16936, 141: 1.16901, 142: 1.16667, 143: 1.16667, 144: 1.16418, 145: 1.16216, 146: 1.16153, 147: 1.16, 148: 1.15942, 149: 1.15714, 150: 1.15714, 151: 1.15584, 152: 1.15385, 153: 1.15152, 154: 1.15068, 155: 1.14971, 156: 1.14815, 157: 1.14493, 158: 1.14466, 159: 1.14474, 160: 1.14286, 161: 1.14286, 162: 1.14286, 163: 1.14124, 164: 1.14103, 165: 1.13889, 166: 1.13924, 167: 1.13953, 168: 1.1375, 169: 1.13483, 170: 1.13514, 171: 1.13415, 172: 1.13293, 173: 1.13374, 174: 1.13253, 175: 1.13095, 176: 1.13095, 177: 1.13095, 178: 1.12964, 179: 1.12941, 180: 1.12903, 181: 1.12766, 182: 1.12644, 183: 1.12632, 184: 1.12365, 185: 1.12255, 186: 1.12195, 187: 1.12158, 188: 1.12195, 189: 1.12048, 190: 1.11978, 191: 1.11828, 192: 1.11828, 193: 1.11702, 194: 1.11702, 195: 1.1165, 196: 1.11458, 197: 1.11458, 198: 1.11416, 199: 1.1134, 200: 1.11224, 201: 1.11111, 202: 1.10891, 203: 1.1087, 204: 1.1087, 205: 1.10769, 206: 1.10753, 207: 1.10714, 208: 1.10638, 209: 1.10659, 210: 1.10526, 211: 1.10501, 212: 1.1028, 213: 1.10185, 214: 1.10101, 215: 1.10046, 216: 1.10092, 217: 1.1, 218: 1.1, 219: 1.09901, 220: 1.09769, 221: 1.09709, 222: 1.09709, 223: 1.09649, 224: 1.09804, 225: 1.09821, 226: 1.09735, 227: 1.09709, 228: 1.09524, 229: 1.09524, 230: 1.09402, 231: 1.09524, 232: 1.09524, 233: 1.09524, 234: 1.09524, 235: 1.09483, 236: 1.09434, 237: 1.09322, 238: 1.09244, 239: 1.09167, 240: 1.09167, 241: 1.09167, 242: 1.09091, 243: 1.09016, 244: 1.08943, 245: 1.08871, 246: 1.08871, 247: 1.08871, 248: 1.088, 249: 1.0885, 250: 1.0885, 251: 1.08759, 252: 1.08759, 253: 1.08713, 254: 1.08661, 255: 1.08627, 256: 1.08462, 257: 1.08475, 258: 1.08451, 259: 1.08462, 260: 1.08462, 261: 1.08462, 262: 1.08397, 263: 1.08403, 264: 1.08397, 265: 1.08397, 266: 1.08333, 267: 1.08271, 268: 1.08197, 269: 1.08148, 270: 1.08088, 271: 1.08029, 272: 1.08029, 273: 1.07971, 274: 1.07971, 275: 1.07914, 276: 1.07874, 277: 1.0785, 278: 1.07792, 279: 1.07746, 280: 1.07749, 281: 1.07692, 282: 1.07634, 283: 1.07576, 284: 1.07561, 285: 1.07519, 286: 1.07458, 287: 1.07483, 288: 1.07407, 289: 1.07368, 290: 1.07383, 291: 1.07343, 292: 1.07353, 293: 1.07299, 294: 1.07285, 295: 1.07333, 296: 1.07285, 297: 1.07237, 298: 1.0719, 299: 1.07143, 300: 1.07143, 301: 1.07143, 302: 1.07143, 303: 1.07143, 304: 1.07055, 305: 1.07047, 306: 1.06993, 307: 1.06962, 308: 1.06918, 309: 1.06897, 310: 1.06832, 311: 1.06841, 312: 1.06748, 313: 1.0673, 314: 1.06803, 315: 1.06748, 316: 1.0679, 317: 1.06667, 318: 1.06667, 319: 1.06627, 320: 1.06605, 321: 1.06579, 322: 1.06542, 323: 1.06542, 324: 1.06522, 325: 1.06536, 326: 1.06471, 327: 1.06494, 328: 1.06548, 329: 1.06542, 330: 1.06501, 331: 1.06452, 332: 1.06395, 333: 1.06358, 334: 1.06322, 335: 1.06358, 336: 1.06358, 337: 1.0625, 338: 1.06286, 339: 1.0625, 340: 1.06215, 341: 1.06215, 342: 1.06215, 343: 1.06173, 344: 1.0618, 345: 1.06145, 346: 1.06111, 347: 1.06098, 348: 1.06044, 349: 1.06044, 350: 1.06011, 351: 1.06044, 352: 1.06034, 353: 1.06011, 354: 1.06005, 355: 1.05929, 356: 1.05913, 357: 1.05851, 358: 1.05882, 359: 1.05882, 360: 1.05851, 361: 1.05823, 362: 1.0582, 363: 1.05759, 364: 1.05753, 365: 1.05714, 366: 1.05738, 367: 1.05682, 368: 1.05714, 369: 1.0566, 370: 1.05587, 371: 1.05588, 372: 1.05587, 373: 1.05556, 374: 1.05584, 375: 1.05523, 376: 1.05525, 377: 1.055, 378: 1.05556, 379: 1.055, 380: 1.05512, 381: 1.05435, 382: 1.05405, 383: 1.05427, 384: 1.05405, 385: 1.05405, 386: 1.05376, 387: 1.05419, 388: 1.05419, 389: 1.05376, 390: 1.05319, 391: 1.05319, 392: 1.05288, 393: 1.05238, 394: 1.05213, 395: 1.0519, 396: 1.05213, 397: 1.05208, 398: 1.05155, 399: 1.05199, 400: 1.05181, 401: 1.05181, 402: 1.05102, 403: 1.0514, 404: 1.05122, 405: 1.05069, 406: 1.05046, 407: 1.05063, 408: 1.05073, 409: 1.05023, 410: 1.05024, 411: 1.04975, 412: 1.04955, 413: 1.04955, 414: 1.04926, 415: 1.04926, 416: 1.0495, 417: 1.04878, 418: 1.04854, 419: 1.04883, 420: 1.04819, 421: 1.04803, 422: 1.04785, 423: 1.04785, 424: 1.04762, 425: 1.04762, 426: 1.04762, 427: 1.04717, 428: 1.04717, 429: 1.04681, 430: 1.04688, 431: 1.04673, 432: 1.04695, 433: 1.04695, 434: 1.04668, 435: 1.04641, 436: 1.04587, 437: 1.04597, 438: 1.04587, 439: 1.04566, 440: 1.04525, 441: 1.04464, 442: 1.04497, 443: 1.04468, 444: 1.04484, 445: 1.04464, 446: 1.04472, 447: 1.04484, 448: 1.04464, 449: 1.04484, 450: 1.04453, 451: 1.0444, 452: 1.04425, 453: 1.04393, 454: 1.04348, 455: 1.04328, 456: 1.04283, 457: 1.04286, 458: 1.04286, 459: 1.04237, 460: 1.04206, 461: 1.04185, 462: 1.04202, 463: 1.04183, 464: 1.04202, 465: 1.04222, 466: 1.04202, 467: 1.042, 468: 1.04191, 469: 1.04159, 470: 1.04167, 471: 1.04132, 472: 1.04115, 473: 1.04065, 474: 1.04032, 475: 1.04032, 476: 1.04042, 477: 1.04016, 478: 1.04016, 479: 1.03975, 480: 1.03968, 481: 1.03967, 482: 1.03937, 483: 1.03878, 484: 1.03896, 485: 1.03876, 486: 1.03862, 487: 1.03846, 488: 1.03788, 489: 1.03861, 490: 1.03815, 491: 1.03788, 492: 1.03782, 493: 1.03788, 494: 1.03788, 495: 1.0377, 496: 1.03774, 497: 1.03748, 498: 1.03745, 499: 1.03717, 500: 1.03717, 501: 1.0365, 502: 1.03704, 503: 1.03654, 504: 1.03636, 505: 1.03587, 506: 1.03626, 507: 1.03585, 508: 1.03593, 509: 1.03578, 510: 1.03604, 511: 1.03578, 512: 1.03543, 513: 1.03521, 514: 1.03509, 515: 1.03512, 516: 1.03462, 517: 1.03435, 518: 1.03462, 519: 1.03422, 520: 1.03435, 521: 1.03383, 522: 1.03383, 523: 1.03383, 524: 1.03364, 525: 1.03351, 526: 1.03333, 527: 1.03333, 528: 1.03309, 529: 1.03279, 530: 1.03273, 531: 1.03267, 532: 1.03232, 533: 1.03249, 534: 1.03196, 535: 1.03203, 536: 1.03196, 537: 1.03189, 538: 1.03203, 539: 1.0318, 540: 1.03137, 541: 1.0312, 542: 1.03093, 543: 1.03077, 544: 1.03051, 545: 1.02996, 546: 1.02977, 547: 1.02962, 548: 1.0298, 549: 1.02941, 550: 1.02974, 551: 1.02963, 552: 1.02951, 553: 1.02953, 554: 1.02963, 555: 1.02936, 556: 1.02941, 557: 1.0293, 558: 1.02914, 559: 1.02888, 560: 1.02901, 561: 1.02899, 562: 1.02878, 563: 1.02878, 564: 1.02883, 565: 1.02931, 566: 1.02867, 567: 1.02857, 568: 1.02842, 569: 1.02857, 570: 1.02847, 571: 1.02847, 572: 1.02838, 573: 1.02787, 574: 1.02807, 575: 1.02817, 576: 1.02787, 577: 1.02759, 578: 1.02759, 579: 1.02768, 580: 1.02759, 581: 1.02749, 582: 1.02749, 583: 1.02757, 584: 1.0274, 585: 1.0273, 586: 1.02721, 587: 1.02716, 588: 1.02721, 589: 1.02694, 590: 1.02676, 591: 1.02698, 592: 1.0265, 593: 1.02613, 594: 1.02642, 595: 1.02672, 596: 1.02649, 597: 1.0264, 598: 1.02642, 599: 1.02641, 600: 1.02632, 601: 1.02613, 602: 1.02623, 603: 1.02619, 604: 1.02597, 605: 1.02581, 606: 1.02556, 607: 1.02556, 608: 1.02564, 609: 1.02564, 610: 1.02572, 611: 1.02614, 612: 1.02618, 613: 1.02577, 614: 1.02552, 615: 1.02547, 616: 1.02543, 617: 1.02548, 618: 1.025, 619: 1.02509, 620: 1.02484, 621: 1.0252, 622: 1.02488, 623: 1.02467, 624: 1.02465, 625: 1.02446, 626: 1.02469, 627: 1.02465, 628: 1.02469, 629: 1.02439, 630: 1.02448, 631: 1.02439, 632: 1.02412, 633: 1.02417, 634: 1.0241, 635: 1.02385, 636: 1.02392, 637: 1.02373, 638: 1.02385, 639: 1.02414, 640: 1.02392, 641: 1.02404, 642: 1.02389, 643: 1.02362, 644: 1.0234, 645: 1.02336, 646: 1.02326, 647: 1.02336, 648: 1.02315, 649: 1.02265, 650: 1.02281, 651: 1.02288, 652: 1.02276, 653: 1.02244, 654: 1.02247, 655: 1.02229, 656: 1.02244, 657: 1.02287, 658: 1.02276, 659: 1.02289, 660: 1.0228, 661: 1.0228, 662: 1.0228, 663: 1.0228, 664: 1.02236, 665: 1.02236, 666: 1.02229, 667: 1.02236, 668: 1.02239, 669: 1.02244, 670: 1.02215, 671: 1.02203, 672: 1.02201, 673: 1.02201, 674: 1.02208, 675: 1.02201, 676: 1.02188, 677: 1.02201, 678: 1.02194, 679: 1.02168, 680: 1.02154, 681: 1.02154, 682: 1.02134, 683: 1.02128, 684: 1.02128, 685: 1.02121, 686: 1.02124, 687: 1.02141, 688: 1.02137, 689: 1.0216, 690: 1.02149, 691: 1.0216, 692: 1.02118, 693: 1.02121, 694: 1.02121, 695: 1.02118, 696: 1.02108, 697: 1.02116, 698: 1.02124, 699: 1.02134, 700: 1.0216, 701: 1.02154, 702: 1.02149, 703: 1.02141, 704: 1.02128, 705: 1.02141, 706: 1.02115, 707: 1.02124, 708: 1.02096, 709: 1.02115, 710: 1.0209, 711: 1.0206, 712: 1.02056, 713: 1.02067, 714: 1.02101, 715: 1.02115, 716: 1.02107, 717: 1.02086, 718: 1.0209, 719: 1.02083, 720: 1.02083, 721: 1.02054, 722: 1.02029, 723: 1.02043, 724: 1.02029, 725: 1.02029, 726: 1.02006, 727: 1.02006, 728: 1.01961, 729: 1.01983, 730: 1.01977, 731: 1.01983, 732: 1.01982, 733: 1.02, 734: 1.02, 735: 1.01983, 736: 1.01973, 737: 1.01977, 738: 1.01991, 739: 1.02006, 740: 1.02, 741: 1.01989, 742: 1.02, 743: 1.01977, 744: 1.01939, 745: 1.01942, 746: 1.01969, 747: 1.01942, 748: 1.01944, 749: 1.01942, 750: 1.01934, 751: 1.01934, 752: 1.01905, 753: 1.01931, 754: 1.01895, 755: 1.01887, 756: 1.01887, 757: 1.01903, 758: 1.01884, 759: 1.01884, 760: 1.01857, 761: 1.01857, 762: 1.01837, 763: 1.01857, 764: 1.01852, 765: 1.0186, 766: 1.01847, 767: 1.01862, 768: 1.01857, 769: 1.01858, 770: 1.01846, 771: 1.01828, 772: 1.01829, 773: 1.01837, 774: 1.01842, 775: 1.01841, 776: 1.01837, 777: 1.01836, 778: 1.01846, 779: 1.01828, 780: 1.01809, 781: 1.01798, 782: 1.01804, 783: 1.01823, 784: 1.0184, 785: 1.01828, 786: 1.01823, 787: 1.01811, 788: 1.01818, 789: 1.01818, 790: 1.01786, 791: 1.01791, 792: 1.01768, 793: 1.01749, 794: 1.01752, 795: 1.01754, 796: 1.01765, 797: 1.01747, 798: 1.01764, 799: 1.01743, 800: 1.01772, 801: 1.01781, 802: 1.01776, 803: 1.01757, 804: 1.01754, 805: 1.01739, 806: 1.01705, 807: 1.0169, 808: 1.01688, 809: 1.01693, 810: 1.01695, 811: 1.01691, 812: 1.01714, 813: 1.01719, 814: 1.01715, 815: 1.01709, 816: 1.01731, 817: 1.01722, 818: 1.01741, 819: 1.01737, 820: 1.01724, 821: 1.01706, 822: 1.01709, 823: 1.01708, 824: 1.017, 825: 1.01674, 826: 1.01674, 827: 1.01674, 828: 1.01653, 829: 1.01665, 830: 1.01686, 831: 1.01647, 832: 1.01644, 833: 1.01635, 834: 1.01651, 835: 1.01673, 836: 1.0169, 837: 1.01684, 838: 1.01695, 839: 1.01693, 840: 1.01695, 841: 1.01665, 842: 1.01653, 843: 1.01653, 844: 1.0163, 845: 1.01626, 846: 1.01604, 847: 1.01609, 848: 1.01609, 849: 1.01606, 850: 1.01615, 851: 1.01594, 852: 1.016, 853: 1.01592, 854: 1.01567, 855: 1.01579, 856: 1.01571, 857: 1.01575, 858: 1.01567, 859: 1.01552, 860: 1.01556, 861: 1.01542, 862: 1.01544, 863: 1.01558, 864: 1.01563, 865: 1.01558, 866: 1.01538, 867: 1.01531, 868: 1.01531, 869: 1.01534, 870: 1.01527, 871: 1.01542, 872: 1.01538, 873: 1.01538, 874: 1.01529, 875: 1.01538, 876: 1.01517, 877: 1.01535, 878: 1.01542, 879: 1.01532, 880: 1.01546, 881: 1.01523, 882: 1.01538, 883: 1.0153, 884: 1.01546, 885: 1.01542, 886: 1.01523, 887: 1.01519, 888: 1.01504, 889: 1.01494, 890: 1.01502, 891: 1.015, 892: 1.01483, 893: 1.01483, 894: 1.01485, 895: 1.01472, 896: 1.01476, 897: 1.01474, 898: 1.01474, 899: 1.0146, 900: 1.0146, 901: 1.01474, 902: 1.01462, 903: 1.01465, 904: 1.01453, 905: 1.01446, 906: 1.01446, 907: 1.01438, 908: 1.01434, 909: 1.01438, 910: 1.01442, 911: 1.01439, 912: 1.01442, 913: 1.01434, 914: 1.01439, 915: 1.01439, 916: 1.01439, 917: 1.01439, 918: 1.01429, 919: 1.01449, 920: 1.01432, 921: 1.01418, 922: 1.0142, 923: 1.01418, 924: 1.01422, 925: 1.01418, 926: 1.01415, 927: 1.01418, 928: 1.01418, 929: 1.01429, 930: 1.01402, 931: 1.01415, 932: 1.01412, 933: 1.01412, 934: 1.01417, 935: 1.01408, 936: 1.01417, 937: 1.01418, 938: 1.01408, 939: 1.01386, 940: 1.01382, 941: 1.01392, 942: 1.01394, 943: 1.01391, 944: 1.01382, 945: 1.01379, 946: 1.01378, 947: 1.01381, 948: 1.01385, 949: 1.01395, 950: 1.01379, 951: 1.0137, 952: 1.01376, 953: 1.01382, 954: 1.01376, 955: 1.01361, 956: 1.01375, 957: 1.01395, 958: 1.01402, 959: 1.01382, 960: 1.01386, 961: 1.01389, 962: 1.0137, 963: 1.01373, 964: 1.01371, 965: 1.01376, 966: 1.01379, 967: 1.01373, 968: 1.01368, 969: 1.01357, 970: 1.01327, 971: 1.01341, 972: 1.01341, 973: 1.01344, 974: 1.01345, 975: 1.01335, 976: 1.01339, 977: 1.01348, 978: 1.01338, 979: 1.01342, 980: 1.01344, 981: 1.01351, 982: 1.01323, 983: 1.01325, 984: 1.01327, 985: 1.0131, 986: 1.01329, 987: 1.01325, 988: 1.01307, 989: 1.01329, 990: 1.01319, 991: 1.01322, 992: 1.01303, 993: 1.0131, 994: 1.01311, 995: 1.01297, 996: 1.01281, 997: 1.01307, 998: 1.01309, 999: 1.01309, 1000: 1.0129, 1001: 1.01292, 1002: 1.01277, 1003: 1.01277, 1004: 1.01285, 1005: 1.01277, 1006: 1.01274, 1007: 1.01259, 1008: 1.01244, 1009: 1.01263, 1010: 1.01233, 1011: 1.01266, 1012: 1.01259, 1013: 1.0127, 1014: 1.01266, 1015: 1.01247, 1016: 1.01263, 1017: 1.01263, 1018: 1.01264, 1019: 1.01254, 1020: 1.01263, 1021: 1.01267, 1022: 1.01255, 1023: 1.01239, 1024: 1.01255, 1025: 1.01256, 1026: 1.01229, 1027: 1.01221, 1028: 1.01253, 1029: 1.01247, 1030: 1.01277, 1031: 1.01253, 1032: 1.01222, 1033: 1.01253, 1034: 1.01251, 1035: 1.01257, 1036: 1.0127, 1037: 1.0126, 1038: 1.01269, 1039: 1.01253, 1040: 1.01266, 1041: 1.01241, 1042: 1.0125, 1043: 1.01253, 1044: 1.01247, 1045: 1.01257, 1046: 1.01243, 1047: 1.0123, 1048: 1.01239, 1049: 1.01261, 1050: 1.01243, 1051: 1.01246, 1052: 1.01227, 1053: 1.01223, 1054: 1.0123, 1055: 1.01211, 1056: 1.01218, 1057: 1.01214, 1058: 1.01195, 1059: 1.01192, 1060: 1.01172, 1061: 1.01181, 1062: 1.01166, 1063: 1.01179, 1064: 1.01192, 1065: 1.01193, 1066: 1.0121, 1067: 1.01203, 1068: 1.01194, 1069: 1.01196, 1070: 1.01208, 1071: 1.01213, 1072: 1.01208, 1073: 1.01188, 1074: 1.01218, 1075: 1.01196, 1076: 1.01198, 1077: 1.01184, 1078: 1.01179, 1079: 1.01164, 1080: 1.01167, 1081: 1.01157, 1082: 1.01142, 1083: 1.01139, 1084: 1.01138, 1085: 1.01138, 1086: 1.01147, 1087: 1.01138, 1088: 1.01143, 1089: 1.01125, 1090: 1.01133, 1091: 1.01135, 1092: 1.01127, 1093: 1.01133, 1094: 1.01119, 1095: 1.01115, 1096: 1.0113, 1097: 1.01139, 1098: 1.01122, 1099: 1.01124, 1100: 1.0114, 1101: 1.01144, 1102: 1.01144, 1103: 1.01144, 1104: 1.01145, 1105: 1.01134, 1106: 1.0114, 1107: 1.01139, 1108: 1.01133, 1109: 1.01129, 1110: 1.01136, 1111: 1.01131, 1112: 1.01139, 1113: 1.01135, 1114: 1.01106, 1115: 1.0111, 1116: 1.01109, 1117: 1.01117, 1118: 1.01116, 1119: 1.01121, 1120: 1.01111, 1121: 1.01112, 1122: 1.01106, 1123: 1.01119, 1124: 1.01116, 1125: 1.01104, 1126: 1.01078, 1127: 1.01085, 1128: 1.01073, 1129: 1.01081, 1130: 1.01079, 1131: 1.01078, 1132: 1.01099, 1133: 1.01086, 1134: 1.01092, 1135: 1.01091, 1136: 1.01082, 1137: 1.01088, 1138: 1.01086, 1139: 1.01088, 1140: 1.01082, 1141: 1.01071, 1142: 1.01065, 1143: 1.01065, 1144: 1.01081, 1145: 1.01082, 1146: 1.01075, 1147: 1.0107, 1148: 1.01076, 1149: 1.01056, 1150: 1.01074, 1151: 1.01071, 1152: 1.01065, 1153: 1.01055, 1154: 1.0107, 1155: 1.01064, 1156: 1.01066, 1157: 1.01065, 1158: 1.01064, 1159: 1.01064, 1160: 1.01057, 1161: 1.01057, 1162: 1.01049, 1163: 1.01033, 1164: 1.01029, 1165: 1.0103, 1166: 1.01035, 1167: 1.01038, 1168: 1.0104, 1169: 1.01031, 1170: 1.01031, 1171: 1.01043, 1172: 1.01031, 1173: 1.01026, 1174: 1.01033, 1175: 1.01036, 1176: 1.0103, 1177: 1.0103, 1178: 1.0103, 1179: 1.01046, 1180: 1.01046, 1181: 1.01046, 1182: 1.01045, 1183: 1.01049, 1184: 1.01045, 1185: 1.01044, 1186: 1.01034, 1187: 1.01029, 1188: 1.01016, 1189: 1.0102, 1190: 1.01019, 1191: 1.01031, 1192: 1.01031, 1193: 1.01029, 1194: 1.01018, 1195: 1.01018, 1196: 1.01015, 1197: 1.01018, 1198: 1.01012, 1199: 1.00997, 1200: 1.01004, 1201: 1.00986, 1202: 1.00997, 1203: 1.00999, 1204: 1.00998, 1205: 1.00997, 1206: 1.00994, 1207: 1.00992, 1208: 1.00993, 1209: 1.01003, 1210: 1.00998, 1211: 1.00998, 1212: 1.00996, 1213: 1.00998, 1214: 1.01004, 1215: 1.0098, 1216: 1.00978, 1217: 1.00978, 1218: 1.0097, 1219: 1.0095, 1220: 1.00948, 1221: 1.00945, 1222: 1.00945, 1223: 1.00952, 1224: 1.00965, 1225: 1.00962, 1226: 1.00948, 1227: 1.00954, 1228: 1.00946, 1229: 1.00955, 1230: 1.00961, 1231: 1.00962, 1232: 1.00948, 1233: 1.00957, 1234: 1.00953, 1235: 1.00959, 1236: 1.00957, 1237: 1.00971, 1238: 1.00966, 1239: 1.00978, 1240: 1.00973, 1241: 1.00953, 1242: 1.00945, 1243: 1.00957, 1244: 1.0096, 1245: 1.00955, 1246: 1.00949, 1247: 1.00956, 1248: 1.0096, 1249: 1.00957, 1250: 1.00935, 1251: 1.00923, 1252: 1.00949, 1253: 1.00952, 1254: 1.00945, 1255: 1.00935, 1256: 1.00925, 1257: 1.00926, 1258: 1.0093, 1259: 1.00934, 1260: 1.00917, 1261: 1.00933, 1262: 1.00933, 1263: 1.00933, 1264: 1.00933, 1265: 1.00933, 1266: 1.00952, 1267: 1.00948, 1268: 1.0095, 1269: 1.00943, 1270: 1.00937, 1271: 1.00935, 1272: 1.00943, 1273: 1.00945, 1274: 1.00946, 1275: 1.0094, 1276: 1.00941, 1277: 1.00952, 1278: 1.00962, 1279: 1.00951, 1280: 1.00954, 1281: 1.00952, 1282: 1.00947, 1283: 1.00935, 1284: 1.00924, 1285: 1.00933, 1286: 1.00918, 1287: 1.00897, 1288: 1.00911, 1289: 1.00909, 1290: 1.00885, 1291: 1.00882, 1292: 1.00847, 1293: 1.00826, 1294: 1.00826, 1295: 1.00827, 1296: 1.00829, 1297: 1.00857, 1298: 1.00847, 1299: 1.00846, 1300: 1.00829, 1301: 1.00829, 1302: 1.00826, 1303: 1.00818, 1304: 1.00811, 1305: 1.00804, 1306: 1.00802, 1307: 1.00794, 1308: 1.00791, 1309: 1.00794, 1310: 1.00808, 1311: 1.00807, 1312: 1.00811, 1313: 1.00811, 1314: 1.00803, 1315: 1.00801, 1316: 1.00808, 1317: 1.00811, 1318: 1.00806, 1319: 1.00802, 1320: 1.00801, 1321: 1.00792, 1322: 1.00802, 1323: 1.00813, 1324: 1.00813, 1325: 1.00815, 1326: 1.00817, 1327: 1.00821, 1328: 1.00836, 1329: 1.0082, 1330: 1.00819, 1331: 1.00821, 1332: 1.00816, 1333: 1.00816, 1334: 1.00815, 1335: 1.00818, 1336: 1.00819, 1337: 1.00814, 1338: 1.00806, 1339: 1.00805, 1340: 1.00795, 1341: 1.00793, 1342: 1.00784, 1343: 1.00787, 1344: 1.00791, 1345: 1.0079, 1346: 1.00784, 1347: 1.00784, 1348: 1.00776, 1349: 1.00784, 1350: 1.00784, 1351: 1.00775, 1352: 1.00774, 1353: 1.00774, 1354: 1.00771, 1355: 1.00778, 1356: 1.00777, 1357: 1.00774, 1358: 1.00777, 1359: 1.00778, 1360: 1.00777, 1361: 1.00771, 1362: 1.00777, 1363: 1.0078, 1364: 1.00773, 1365: 1.00763, 1366: 1.00771, 1367: 1.00774, 1368: 1.00767, 1369: 1.00762, 1370: 1.00766, 1371: 1.0076, 1372: 1.00766, 1373: 1.00769, 1374: 1.00765, 1375: 1.00766, 1376: 1.00773, 1377: 1.00763, 1378: 1.00777, 1379: 1.00766, 1380: 1.00766, 1381: 1.00768, 1382: 1.00766, 1383: 1.00763, 1384: 1.0076, 1385: 1.00761, 1386: 1.00761, 1387: 1.00763, 1388: 1.00768, 1389: 1.0076, 1390: 1.0076, 1391: 1.00762, 1392: 1.00766, 1393: 1.0077, 1394: 1.00775, 1395: 1.00774, 1396: 1.00764, 1397: 1.00766, 1398: 1.00772, 1399: 1.00769, 1400: 1.0076, 1401: 1.00756, 1402: 1.00758, 1403: 1.00759, 1404: 1.00756, 1405: 1.00756, 1406: 1.0076, 1407: 1.00758, 1408: 1.00756, 1409: 1.0076, 1410: 1.00768, 1411: 1.0076, 1412: 1.00763, 1413: 1.00762, 1414: 1.00757, 1415: 1.00757, 1416: 1.00749, 1417: 1.00746, 1418: 1.00746, 1419: 1.00755, 1420: 1.00761, 1421: 1.00755, 1422: 1.00755, 1423: 1.0075, 1424: 1.00745, 1425: 1.00735, 1426: 1.00726, 1427: 1.00734, 1428: 1.00733, 1429: 1.00739, 1430: 1.00739, 1431: 1.00731, 1432: 1.00732, 1433: 1.00728, 1434: 1.00727, 1435: 1.00723, 1436: 1.0073, 1437: 1.00727, 1438: 1.00725, 1439: 1.00721, 1440: 1.00725, 1441: 1.00731, 1442: 1.00727, 1443: 1.00731, 1444: 1.00731, 1445: 1.00725, 1446: 1.00722, 1447: 1.00721, 1448: 1.00731, 1449: 1.00735, 1450: 1.00735, 1451: 1.00733, 1452: 1.00735, 1453: 1.00734, 1454: 1.00728, 1455: 1.00725, 1456: 1.0073, 1457: 1.00728, 1458: 1.00731, 1459: 1.00723, 1460: 1.00714, 1461: 1.00701, 1462: 1.00707, 1463: 1.007, 1464: 1.00703, 1465: 1.00695, 1466: 1.00698, 1467: 1.00701, 1468: 1.00701, 1469: 1.00692, 1470: 1.007, 1471: 1.00698, 1472: 1.00698, 1473: 1.00697, 1474: 1.00695, 1475: 1.00699, 1476: 1.00692, 1477: 1.007, 1478: 1.00692, 1479: 1.00704, 1480: 1.00704, 1481: 1.00712, 1482: 1.0071, 1483: 1.00707, 1484: 1.00715, 1485: 1.00707, 1486: 1.00707, 1487: 1.00713, 1488: 1.00714, 1489: 1.00714, 1490: 1.00703, 1491: 1.00707, 1492: 1.00712, 1493: 1.00708, 1494: 1.00699, 1495: 1.00708, 1496: 1.00703, 1497: 1.00705, 1498: 1.00701, 1499: 1.00693, 1500: 1.00696, 1501: 1.00697, 1502: 1.00694, 1503: 1.00686, 1504: 1.00689, 1505: 1.0069, 1506: 1.00686, 1507: 1.00694, 1508: 1.00694, 1509: 1.00696, 1510: 1.00698, 1511: 1.00696, 1512: 1.00701, 1513: 1.00697, 1514: 1.00701, 1515: 1.00701, 1516: 1.00711, 1517: 1.00709, 1518: 1.00704, 1519: 1.00707, 1520: 1.00703, 1521: 1.00703, 1522: 1.00701, 1523: 1.00707, 1524: 1.00704, 1525: 1.00698, 1526: 1.00694, 1527: 1.00691, 1528: 1.00697, 1529: 1.00693, 1530: 1.00696, 1531: 1.00689, 1532: 1.00697, 1533: 1.00697, 1534: 1.00695, 1535: 1.00696, 1536: 1.00695, 1537: 1.00694, 1538: 1.00695, 1539: 1.00689, 1540: 1.00694, 1541: 1.00693, 1542: 1.00687, 1543: 1.00687, 1544: 1.00683, 1545: 1.00672, 1546: 1.00673, 1547: 1.00673, 1548: 1.00675, 1549: 1.0069, 1550: 1.00678, 1551: 1.00678, 1552: 1.00678, 1553: 1.00677, 1554: 1.00676, 1555: 1.0068, 1556: 1.00667, 1557: 1.00672, 1558: 1.00668, 1559: 1.00672, 1560: 1.00667, 1561: 1.00664, 1562: 1.00673, 1563: 1.00686, 1564: 1.00684, 1565: 1.00685, 1566: 1.00682, 1567: 1.00677, 1568: 1.0068, 1569: 1.00673, 1570: 1.00665, 1571: 1.00669, 1572: 1.00669, 1573: 1.00669, 1574: 1.00675, 1575: 1.00672, 1576: 1.00665, 1577: 1.00647, 1578: 1.00644, 1579: 1.00598, 1580: 1.00598, 1581: 1.00598, 1582: 1.00617, 1583: 1.00639, 1584: 1.00655, 1585: 1.00654, 1586: 1.00636, 1587: 1.00651, 1588: 1.00656, 1589: 1.0065, 1590: 1.00641, 1591: 1.00631, 1592: 1.00641, 1593: 1.00664, 1594: 1.00663, 1595: 1.0066, 1596: 1.0066, 1597: 1.00635, 1598: 1.00656, 1599: 1.00656, 1600: 1.00663, 1601: 1.00658, 1602: 1.00668, 1603: 1.00664, 1604: 1.00661, 1605: 1.00665, 1606: 1.00667, 1607: 1.00659, 1608: 1.00656, 1609: 1.00655, 1610: 1.00641, 1611: 1.00593, 1612: 1.00595, 1613: 1.00587, 1614: 1.00598, 1615: 1.00574, 1616: 1.00557, 1617: 1.0057, 1618: 1.0056, 1619: 1.00552, 1620: 1.00556, 1621: 1.00556, 1622: 1.00556, 1623: 1.00565, 1624: 1.00561, 1625: 1.00563, 1626: 1.00575, 1627: 1.00561, 1628: 1.00562, 1629: 1.00561, 1630: 1.00554, 1631: 1.00555, 1632: 1.00565, 1633: 1.00554, 1634: 1.00548, 1635: 1.00555, 1636: 1.0056, 1637: 1.00555, 1638: 1.00565, 1639: 1.00569, 1640: 1.00575, 1641: 1.0057, 1642: 1.0056, 1643: 1.00559, 1644: 1.00556, 1645: 1.00567, 1646: 1.00599, 1647: 1.00581, 1648: 1.00581, 1649: 1.00574, 1650: 1.00602, 1651: 1.00616, 1652: 1.00623, 1653: 1.0062, 1654: 1.00613, 1655: 1.00643, 1656: 1.00638, 1657: 1.00637, 1658: 1.00638, 1659: 1.00628, 1660: 1.00632, 1661: 1.00642, 1662: 1.00632, 1663: 1.00605, 1664: 1.00601, 1665: 1.00605, 1666: 1.00583, 1667: 1.00566, 1668: 1.00576, 1669: 1.00564, 1670: 1.00575, 1671: 1.00575, 1672: 1.00578, 1673: 1.00588, 1674: 1.00561, 1675: 1.00555, 1676: 1.0055, 1677: 1.00542, 1678: 1.00538, 1679: 1.00538, 1680: 1.00547, 1681: 1.00538, 1682: 1.00534, 1683: 1.00532, 1684: 1.00531, 1685: 1.00531, 1686: 1.00534, 1687: 1.00532, 1688: 1.00538, 1689: 1.00533, 1690: 1.00535, 1691: 1.00527, 1692: 1.00522, 1693: 1.00522, 1694: 1.00524, 1695: 1.00521, 1696: 1.00525, 1697: 1.00528, 1698: 1.0053, 1699: 1.00528, 1700: 1.00526, 1701: 1.00528, 1702: 1.00528, 1703: 1.00527, 1704: 1.00525, 1705: 1.00528, 1706: 1.00527, 1707: 1.0053, 1708: 1.0053, 1709: 1.0053, 1710: 1.00529, 1711: 1.00528, 1712: 1.00527, 1713: 1.00526, 1714: 1.00522, 1715: 1.00518, 1716: 1.0052, 1717: 1.0052, 1718: 1.00521, 1719: 1.00528, 1720: 1.00529, 1721: 1.00533, 1722: 1.00529, 1723: 1.00532, 1724: 1.00534, 1725: 1.00528, 1726: 1.00535, 1727: 1.00524, 1728: 1.00524, 1729: 1.00521, 1730: 1.0053, 1731: 1.00534, 1732: 1.00534, 1733: 1.00536, 1734: 1.00535, 1735: 1.00537, 1736: 1.00532, 1737: 1.00535, 1738: 1.00529, 1739: 1.00532, 1740: 1.00537, 1741: 1.0053, 1742: 1.00536, 1743: 1.0053, 1744: 1.00529, 1745: 1.00529, 1746: 1.00526, 1747: 1.00521, 1748: 1.0052, 1749: 1.00525, 1750: 1.0052, 1751: 1.00524, 1752: 1.00526, 1753: 1.00524, 1754: 1.0053, 1755: 1.00529, 1756: 1.00526, 1757: 1.00521, 1758: 1.00521, 1759: 1.00525, 1760: 1.0052, 1761: 1.00523, 1762: 1.00522, 1763: 1.00522, 1764: 1.0052, 1765: 1.00512, 1766: 1.00509, 1767: 1.0051, 1768: 1.00508, 1769: 1.00508, 1770: 1.00508, 1771: 1.00512, 1772: 1.00506, 1773: 1.00508, 1774: 1.00508, 1775: 1.00506, 1776: 1.00508, 1777: 1.00505, 1778: 1.00506, 1779: 1.0051, 1780: 1.00506, 1781: 1.00507, 1782: 1.00509, 1783: 1.00514, 1784: 1.00511, 1785: 1.00514, 1786: 1.00512, 1787: 1.00509, 1788: 1.00508, 1789: 1.00513, 1790: 1.00512, 1791: 1.00512, 1792: 1.00505, 1793: 1.00504, 1794: 1.00503, 1795: 1.00499, 1796: 1.00499, 1797: 1.00497, 1798: 1.00492, 1799: 1.00489, 1800: 1.00493, 1801: 1.00491, 1802: 1.00489, 1803: 1.00489, 1804: 1.00491, 1805: 1.00493, 1806: 1.00492, 1807: 1.00494, 1808: 1.00494, 1809: 1.00491, 1810: 1.00493, 1811: 1.00495, 1812: 1.00495, 1813: 1.00495, 1814: 1.00493, 1815: 1.00493, 1816: 1.00493, 1817: 1.00489, 1818: 1.00489, 1819: 1.00487, 1820: 1.00486, 1821: 1.00489, 1822: 1.00489, 1823: 1.00496, 1824: 1.00489, 1825: 1.00486, 1826: 1.00488, 1827: 1.00492, 1828: 1.00491, 1829: 1.00486, 1830: 1.00489, 1831: 1.00486, 1832: 1.00485, 1833: 1.00485, 1834: 1.00486, 1835: 1.00485, 1836: 1.00485, 1837: 1.00489, 1838: 1.00485, 1839: 1.00485, 1840: 1.00491, 1841: 1.00491, 1842: 1.00491, 1843: 1.0049, 1844: 1.00487, 1845: 1.00483, 1846: 1.00486, 1847: 1.00482, 1848: 1.00486, 1849: 1.00483, 1850: 1.00482, 1851: 1.00485, 1852: 1.00483, 1853: 1.00483, 1854: 1.00484, 1855: 1.00482, 1856: 1.0048, 1857: 1.00482, 1858: 1.0048, 1859: 1.00485, 1860: 1.00478, 1861: 1.00476, 1862: 1.00479, 1863: 1.00478, 1864: 1.0048, 1865: 1.00481, 1866: 1.00481, 1867: 1.00484, 1868: 1.0048, 1869: 1.0048, 1870: 1.00485, 1871: 1.00478, 1872: 1.0048, 1873: 1.00478, 1874: 1.0048, 1875: 1.00482, 1876: 1.00479, 1877: 1.00481, 1878: 1.00481, 1879: 1.00482, 1880: 1.0048, 1881: 1.00481, 1882: 1.00481, 1883: 1.00487, 1884: 1.00483, 1885: 1.00483, 1886: 1.00478, 1887: 1.00479, 1888: 1.00477, 1889: 1.00477, 1890: 1.00478, 1891: 1.00474, 1892: 1.00482, 1893: 1.00482, 1894: 1.00481, 1895: 1.00484, 1896: 1.00483, 1897: 1.00483, 1898: 1.00475, 1899: 1.00474, 1900: 1.00479, 1901: 1.00476, 1902: 1.00477, 1903: 1.00475, 1904: 1.00476, 1905: 1.00476, 1906: 1.00477, 1907: 1.00475, 1908: 1.00477, 1909: 1.00476, 1910: 1.00476, 1911: 1.00476, 1912: 1.00473, 1913: 1.00473, 1914: 1.00475, 1915: 1.00469, 1916: 1.00466, 1917: 1.0047, 1918: 1.00471, 1919: 1.00474, 1920: 1.00475, 1921: 1.00478, 1922: 1.00472, 1923: 1.00469, 1924: 1.00467, 1925: 1.00469, 1926: 1.00469, 1927: 1.00466, 1928: 1.00465, 1929: 1.00465, 1930: 1.00461, 1931: 1.00465, 1932: 1.00468, 1933: 1.00468, 1934: 1.00467, 1935: 1.00467, 1936: 1.00465, 1937: 1.00464, 1938: 1.00462, 1939: 1.0046, 1940: 1.00456, 1941: 1.00456, 1942: 1.00453, 1943: 1.00453, 1944: 1.00452, 1945: 1.00451, 1946: 1.00458, 1947: 1.00458, 1948: 1.00453, 1949: 1.00456, 1950: 1.00458, 1951: 1.00452, 1952: 1.00452, 1953: 1.00452, 1954: 1.00454, 1955: 1.00454, 1956: 1.00451, 1957: 1.00452, 1958: 1.00452, 1959: 1.00449, 1960: 1.00449, 1961: 1.00449, 1962: 1.00454, 1963: 1.00457, 1964: 1.00455, 1965: 1.00457, 1966: 1.00458, 1967: 1.00457, 1968: 1.00452, 1969: 1.00456, 1970: 1.00458, 1971: 1.00464, 1972: 1.00463, 1973: 1.00453, 1974: 1.00452, 1975: 1.00457, 1976: 1.00451, 1977: 1.00451, 1978: 1.00454, 1979: 1.00456, 1980: 1.00452, 1981: 1.00447, 1982: 1.00449, 1983: 1.00446, 1984: 1.00445, 1985: 1.00438, 1986: 1.00434, 1987: 1.00444, 1988: 1.00444, 1989: 1.00441, 1990: 1.00439, 1991: 1.00447, 1992: 1.00448, 1993: 1.00448, 1994: 1.00444, 1995: 1.00441, 1996: 1.00439, 1997: 1.00445, 1998: 1.00448, 1999: 1.00445}
        perc_3 = {21: 8.0, 22: 5.0, 23: 3.66667, 24: 3.25, 25: 2.75, 26: 2.5, 27: 2.33333, 28: 2.199, 29: 2.125, 30: 2.0, 31: 1.88889, 32: 1.83333, 33: 1.8, 34: 1.72727, 35: 1.7, 36: 1.66667, 37: 1.63573, 38: 1.58333, 39: 1.57116, 40: 1.53731, 41: 1.5, 42: 1.47059, 43: 1.46154, 44: 1.4375, 45: 1.4375, 46: 1.42857, 47: 1.41652, 48: 1.38889, 49: 1.375, 50: 1.36828, 51: 1.34739, 52: 1.33333, 53: 1.33333, 54: 1.33333, 55: 1.30435, 56: 1.30422, 57: 1.29167, 58: 1.28571, 59: 1.28, 60: 1.28571, 61: 1.27577, 62: 1.26923, 63: 1.25, 64: 1.25, 65: 1.25782, 66: 1.25, 67: 1.24977, 68: 1.23326, 69: 1.2257, 70: 1.21875, 71: 1.21212, 72: 1.21207, 73: 1.21875, 74: 1.21429, 75: 1.21207, 76: 1.20588, 77: 1.2, 78: 1.19442, 79: 1.18743, 80: 1.18414, 81: 1.17241, 82: 1.18175, 83: 1.17647, 84: 1.175, 85: 1.17492, 86: 1.17489, 87: 1.16667, 88: 1.16655, 89: 1.17073, 90: 1.16667, 91: 1.15786, 92: 1.15909, 93: 1.15906, 94: 1.15385, 95: 1.15215, 96: 1.14633, 97: 1.13999, 98: 1.13953, 99: 1.14277, 100: 1.13952, 101: 1.13633, 102: 1.13325, 103: 1.1272, 104: 1.125, 105: 1.12245, 106: 1.11765, 107: 1.12245, 108: 1.12, 109: 1.12245, 110: 1.125, 111: 1.125, 112: 1.12493, 113: 1.12487, 114: 1.11765, 115: 1.12067, 116: 1.12067, 117: 1.11997, 118: 1.11663, 119: 1.1132, 120: 1.11858, 121: 1.11663, 122: 1.1132, 123: 1.11111, 124: 1.11263, 125: 1.11263, 126: 1.10937, 127: 1.10929, 128: 1.10769, 129: 1.10526, 130: 1.10388, 131: 1.1029, 132: 1.10448, 133: 1.10164, 134: 1.10141, 135: 1.09858, 136: 1.09372, 137: 1.0937, 138: 1.0923, 139: 1.09374, 140: 1.09231, 141: 1.09374, 142: 1.09524, 143: 1.09091, 144: 1.09227, 145: 1.09091, 146: 1.0879, 147: 1.08568, 148: 1.08333, 149: 1.08821, 150: 1.08748, 151: 1.08642, 152: 1.0864, 153: 1.0879, 154: 1.08816, 155: 1.08235, 156: 1.08219, 157: 1.08063, 158: 1.08108, 159: 1.08108, 160: 1.08217, 161: 1.08106, 162: 1.08235, 163: 1.08105, 164: 1.08163, 165: 1.08045, 166: 1.07894, 167: 1.0769, 168: 1.07772, 169: 1.07317, 170: 1.07226, 171: 1.07243, 172: 1.07609, 173: 1.07692, 174: 1.07595, 175: 1.075, 176: 1.07592, 177: 1.07402, 178: 1.07442, 179: 1.07214, 180: 1.07364, 181: 1.07143, 182: 1.07229, 183: 1.07224, 184: 1.07447, 185: 1.07292, 186: 1.07229, 187: 1.0707, 188: 1.07057, 189: 1.06896, 190: 1.06724, 191: 1.06816, 192: 1.06816, 193: 1.06861, 194: 1.06667, 195: 1.06667, 196: 1.06522, 197: 1.06481, 198: 1.06316, 199: 1.06316, 200: 1.06362, 201: 1.06305, 202: 1.06303, 203: 1.06383, 204: 1.05939, 205: 1.06, 206: 1.05824, 207: 1.05879, 208: 1.05824, 209: 1.05659, 210: 1.05768, 211: 1.05941, 212: 1.0594, 213: 1.05785, 214: 1.05931, 215: 1.05831, 216: 1.05713, 217: 1.05607, 218: 1.05713, 219: 1.05714, 220: 1.05554, 221: 1.05357, 222: 1.05644, 223: 1.05659, 224: 1.05466, 225: 1.05357, 226: 1.05354, 227: 1.05128, 228: 1.05455, 229: 1.05353, 230: 1.05357, 231: 1.05306, 232: 1.05083, 233: 1.05308, 234: 1.05085, 235: 1.05224, 236: 1.05217, 237: 1.05262, 238: 1.05216, 239: 1.05172, 240: 1.05262, 241: 1.05185, 242: 1.05308, 243: 1.05185, 244: 1.05146, 245: 1.05341, 246: 1.05452, 247: 1.05083, 248: 1.05184, 249: 1.05126, 250: 1.05128, 251: 1.04996, 252: 1.04917, 253: 1.04892, 254: 1.04878, 255: 1.05042, 256: 1.05083, 257: 1.05, 258: 1.04686, 259: 1.04723, 260: 1.04579, 261: 1.04649, 262: 1.048, 263: 1.04838, 264: 1.04795, 265: 1.04794, 266: 1.04958, 267: 1.04761, 268: 1.04892, 269: 1.04827, 270: 1.04728, 271: 1.0473, 272: 1.04698, 273: 1.0473, 274: 1.04698, 275: 1.04615, 276: 1.04545, 277: 1.04574, 278: 1.04545, 279: 1.04444, 280: 1.04458, 281: 1.04443, 282: 1.04378, 283: 1.0443, 284: 1.04402, 285: 1.04254, 286: 1.04374, 287: 1.04348, 288: 1.04285, 289: 1.04255, 290: 1.04166, 291: 1.04255, 292: 1.04166, 293: 1.04166, 294: 1.04138, 295: 1.04138, 296: 1.04053, 297: 1.04054, 298: 1.04027, 299: 1.04027, 300: 1.04109, 301: 1.04053, 302: 1.04054, 303: 1.03974, 304: 1.04106, 305: 1.04216, 306: 1.04185, 307: 1.04, 308: 1.04162, 309: 1.0414, 310: 1.04081, 311: 1.03895, 312: 1.03869, 313: 1.03727, 314: 1.03797, 315: 1.03796, 316: 1.03819, 317: 1.03796, 318: 1.03659, 319: 1.03593, 320: 1.03774, 321: 1.03636, 322: 1.0377, 323: 1.03749, 324: 1.03636, 325: 1.03614, 326: 1.03702, 327: 1.03614, 328: 1.03703, 329: 1.03804, 330: 1.03725, 331: 1.03727, 332: 1.0368, 333: 1.03657, 334: 1.03658, 335: 1.03636, 336: 1.0368, 337: 1.03657, 338: 1.03749, 339: 1.03659, 340: 1.03592, 341: 1.03571, 342: 1.03592, 343: 1.03508, 344: 1.03529, 345: 1.03529, 346: 1.03508, 347: 1.03352, 348: 1.03333, 349: 1.03508, 350: 1.03508, 351: 1.03409, 352: 1.03333, 353: 1.03448, 354: 1.03369, 355: 1.03296, 356: 1.03314, 357: 1.03225, 358: 1.03296, 359: 1.03279, 360: 1.03279, 361: 1.0326, 362: 1.03139, 363: 1.03261, 364: 1.0303, 365: 1.02924, 366: 1.03157, 367: 1.0306, 368: 1.03124, 369: 1.02957, 370: 1.03141, 371: 1.03141, 372: 1.03124, 373: 1.0303, 374: 1.03125, 375: 1.03141, 376: 1.03092, 377: 1.03092, 378: 1.03157, 379: 1.03076, 380: 1.03077, 381: 1.0306, 382: 1.03061, 383: 1.0306, 384: 1.03093, 385: 1.03091, 386: 1.03045, 387: 1.02898, 388: 1.02985, 389: 1.02912, 390: 1.02827, 391: 1.02898, 392: 1.02884, 393: 1.02912, 394: 1.03, 395: 1.0294, 396: 1.02856, 397: 1.0283, 398: 1.02941, 399: 1.02955, 400: 1.02844, 401: 1.02803, 402: 1.02793, 403: 1.0279, 404: 1.02777, 405: 1.02912, 406: 1.02899, 407: 1.02802, 408: 1.02763, 409: 1.02763, 410: 1.02726, 411: 1.02727, 412: 1.02752, 413: 1.02727, 414: 1.02727, 415: 1.02764, 416: 1.02739, 417: 1.02802, 418: 1.02884, 419: 1.02739, 420: 1.02804, 421: 1.02678, 422: 1.02632, 423: 1.02563, 424: 1.02628, 425: 1.02617, 426: 1.02596, 427: 1.02475, 428: 1.02475, 429: 1.02415, 430: 1.0252, 431: 1.02462, 432: 1.02392, 433: 1.02381, 434: 1.02381, 435: 1.02509, 436: 1.02426, 437: 1.02283, 438: 1.02272, 439: 1.02262, 440: 1.02222, 441: 1.02368, 442: 1.02368, 443: 1.02368, 444: 1.02404, 445: 1.02314, 446: 1.02272, 447: 1.02252, 448: 1.02252, 449: 1.02193, 450: 1.02164, 451: 1.02202, 452: 1.02193, 453: 1.02164, 454: 1.02183, 455: 1.02164, 456: 1.02202, 457: 1.02222, 458: 1.02118, 459: 1.02211, 460: 1.02118, 461: 1.02136, 462: 1.02136, 463: 1.02109, 464: 1.02164, 465: 1.02092, 466: 1.02145, 467: 1.02146, 468: 1.02083, 469: 1.02136, 470: 1.02065, 471: 1.02083, 472: 1.02066, 473: 1.02058, 474: 1.02057, 475: 1.02024, 476: 1.02065, 477: 1.02101, 478: 1.01992, 479: 1.01976, 480: 1.01992, 481: 1.01829, 482: 1.01829, 483: 1.01976, 484: 1.0193, 485: 1.01945, 486: 1.01915, 487: 1.01923, 488: 1.01945, 489: 1.01921, 490: 1.01843, 491: 1.01724, 492: 1.01731, 493: 1.01717, 494: 1.01646, 495: 1.01709, 496: 1.01695, 497: 1.01717, 498: 1.01718, 499: 1.01754, 500: 1.01754, 501: 1.01865, 502: 1.01731, 503: 1.0186, 504: 1.01822, 505: 1.01731, 506: 1.0168, 507: 1.01613, 508: 1.01619, 509: 1.01645, 510: 1.01575, 511: 1.01575, 512: 1.01593, 513: 1.01581, 514: 1.01509, 515: 1.01481, 516: 1.0147, 517: 1.01471, 518: 1.01355, 519: 1.01471, 520: 1.01498, 521: 1.01562, 522: 1.01526, 523: 1.01487, 524: 1.01492, 525: 1.01454, 526: 1.01423, 527: 1.01449, 528: 1.01417, 529: 1.01428, 530: 1.01423, 531: 1.01438, 532: 1.01454, 533: 1.01443, 534: 1.01413, 535: 1.01444, 536: 1.0146, 537: 1.0146, 538: 1.01389, 539: 1.01418, 540: 1.0146, 541: 1.01444, 542: 1.01403, 543: 1.01454, 544: 1.01423, 545: 1.01423, 546: 1.01384, 547: 1.01428, 548: 1.01403, 549: 1.01398, 550: 1.01375, 551: 1.01444, 552: 1.01449, 553: 1.01388, 554: 1.01365, 555: 1.01351, 556: 1.01337, 557: 1.01252, 558: 1.01319, 559: 1.01388, 560: 1.01329, 561: 1.01316, 562: 1.01286, 563: 1.01323, 564: 1.01274, 565: 1.0132, 566: 1.01333, 567: 1.01351, 568: 1.01311, 569: 1.01311, 570: 1.01111, 571: 1.01107, 572: 1.01106, 573: 1.0119, 574: 1.01307, 575: 1.01273, 576: 1.01167, 577: 1.01083, 578: 1.01119, 579: 1.01128, 580: 1.01115, 581: 1.01123, 582: 1.01079, 583: 1.01103, 584: 1.01132, 585: 1.01103, 586: 1.01123, 587: 1.01099, 588: 1.01075, 589: 1.01171, 590: 1.01169, 591: 1.01053, 592: 1.01086, 593: 1.01124, 594: 1.0114, 595: 1.01232, 596: 1.01122, 597: 1.01126, 598: 1.01056, 599: 1.01136, 600: 1.01214, 601: 1.01248, 602: 1.01254, 603: 1.01277, 604: 1.01131, 605: 1.01145, 606: 1.01144, 607: 1.01249, 608: 1.01079, 609: 1.01145, 610: 1.01127, 611: 1.01123, 612: 1.01053, 613: 1.01038, 614: 1.01017, 615: 1.01003, 616: 1.01041, 617: 1.01023, 618: 1.01083, 619: 1.01083, 620: 1.01083, 621: 1.01098, 622: 1.01017, 623: 1.01045, 624: 1.01052, 625: 1.01124, 626: 1.01007, 627: 1.01063, 628: 1.0101, 629: 1.01063, 630: 1.01204, 631: 1.01013, 632: 1.01017, 633: 1.01169, 634: 1.01197, 635: 1.01143, 636: 1.01064, 637: 1.0111, 638: 1.01003, 639: 1.00952, 640: 1.00946, 641: 1.00946, 642: 1.00943, 643: 1.00943, 644: 1.00943, 645: 1.00937, 646: 1.00952, 647: 1.00974, 648: 1.00977, 649: 1.00977, 650: 1.00955, 651: 1.00946, 652: 1.00937, 653: 1.00935, 654: 1.00932, 655: 1.00946, 656: 1.00946, 657: 1.00946, 658: 1.00943, 659: 1.00943, 660: 1.00946, 661: 1.00926, 662: 1.00928, 663: 1.00949, 664: 1.00946, 665: 1.00949, 666: 1.00929, 667: 1.00932, 668: 1.00898, 669: 1.0092, 670: 1.00926, 671: 1.00929, 672: 1.00929, 673: 1.00934, 674: 1.00904, 675: 1.00929, 676: 1.00929, 677: 1.00926, 678: 1.0092, 679: 1.00915, 680: 1.0089, 681: 1.0089, 682: 1.00893, 683: 1.00929, 684: 1.00917, 685: 1.00929, 686: 1.00909, 687: 1.00879, 688: 1.00898, 689: 1.00909, 690: 1.00903, 691: 1.00904, 692: 1.00885, 693: 1.00877, 694: 1.00862, 695: 1.00872, 696: 1.00852, 697: 1.00852, 698: 1.00862, 699: 1.00875, 700: 1.0089, 701: 1.00882, 702: 1.00885, 703: 1.00882, 704: 1.00845, 705: 1.00877, 706: 1.00898, 707: 1.00898, 708: 1.00895, 709: 1.00893, 710: 1.00887, 711: 1.00877, 712: 1.00898, 713: 1.00887, 714: 1.0088, 715: 1.00893, 716: 1.0088, 717: 1.00845, 718: 1.0084, 719: 1.00877, 720: 1.0084, 721: 1.00838, 722: 1.0082, 723: 1.00855, 724: 1.00867, 725: 1.00864, 726: 1.00855, 727: 1.00857, 728: 1.00867, 729: 1.00911, 730: 1.00931, 731: 1.00996, 732: 1.00909, 733: 1.0089, 734: 1.0088, 735: 1.00867, 736: 1.00854, 737: 1.00872, 738: 1.00887, 739: 1.0088, 740: 1.00875, 741: 1.00859, 742: 1.00838, 743: 1.00836, 744: 1.00836, 745: 1.00845, 746: 1.00849, 747: 1.00843, 748: 1.00833, 749: 1.00826, 750: 1.00829, 751: 1.00817, 752: 1.00826, 753: 1.00811, 754: 1.00798, 755: 1.00804, 756: 1.00811, 757: 1.00811, 758: 1.00817, 759: 1.0082, 760: 1.00811, 761: 1.00796, 762: 1.00792, 763: 1.00789, 764: 1.00785, 765: 1.00796, 766: 1.00781, 767: 1.00781, 768: 1.00765, 769: 1.00769, 770: 1.00763, 771: 1.00769, 772: 1.00787, 773: 1.00787, 774: 1.00773, 775: 1.00765, 776: 1.00775, 777: 1.00783, 778: 1.00769, 779: 1.00783, 780: 1.00757, 781: 1.0073, 782: 1.00595, 783: 1.00677, 784: 1.00794, 785: 1.00777, 786: 1.00791, 787: 1.00773, 788: 1.00773, 789: 1.00753, 790: 1.00752, 791: 1.00744, 792: 1.00769, 793: 1.00775, 794: 1.00752, 795: 1.00695, 796: 1.00773, 797: 1.00759, 798: 1.00773, 799: 1.00777, 800: 1.00642, 801: 1.00747, 802: 1.00734, 803: 1.00759, 804: 1.00615, 805: 1.00595, 806: 1.00617, 807: 1.00615, 808: 1.00746, 809: 1.00752, 810: 1.00729, 811: 1.00689, 812: 1.00613, 813: 1.00593, 814: 1.00593, 815: 1.00615, 816: 1.00583, 817: 1.00576, 818: 1.00724, 819: 1.00733, 820: 1.00613, 821: 1.00731, 822: 1.00689, 823: 1.00689, 824: 1.00626, 825: 1.00712, 826: 1.00568, 827: 1.00597, 828: 1.00714, 829: 1.00724, 830: 1.00568, 831: 1.00568, 832: 1.00571, 833: 1.00568, 834: 1.00565, 835: 1.00713, 836: 1.00728, 837: 1.00565, 838: 1.00564, 839: 1.00551, 840: 1.00545, 841: 1.00545, 842: 1.00552, 843: 1.00557, 844: 1.00567, 845: 1.00565, 846: 1.00552, 847: 1.00561, 848: 1.00549, 849: 1.00562, 850: 1.00555, 851: 1.00568, 852: 1.00545, 853: 1.00709, 854: 1.00536, 855: 1.00533, 856: 1.0056, 857: 1.00542, 858: 1.00543, 859: 1.00536, 860: 1.00532, 861: 1.0053, 862: 1.00536, 863: 1.00546, 864: 1.00532, 865: 1.00549, 866: 1.00559, 867: 1.00556, 868: 1.00548, 869: 1.00539, 870: 1.00554, 871: 1.00554, 872: 1.00554, 873: 1.00578, 874: 1.00533, 875: 1.00548, 876: 1.00555, 877: 1.00552, 878: 1.00557, 879: 1.00554, 880: 1.00555, 881: 1.00551, 882: 1.00526, 883: 1.00522, 884: 1.0053, 885: 1.00536, 886: 1.00524, 887: 1.00522, 888: 1.00511, 889: 1.00515, 890: 1.00515, 891: 1.00522, 892: 1.00512, 893: 1.00517, 894: 1.00508, 895: 1.00508, 896: 1.00505, 897: 1.00505, 898: 1.00504, 899: 1.00496, 900: 1.00497, 901: 1.0049, 902: 1.00493, 903: 1.00491, 904: 1.00493, 905: 1.00491, 906: 1.00491, 907: 1.0049, 908: 1.00504, 909: 1.00495, 910: 1.00493, 911: 1.00484, 912: 1.00484, 913: 1.00483, 914: 1.00487, 915: 1.0049, 916: 1.0049, 917: 1.00491, 918: 1.00494, 919: 1.00498, 920: 1.00491, 921: 1.00508, 922: 1.00497, 923: 1.00509, 924: 1.00493, 925: 1.00493, 926: 1.00477, 927: 1.00482, 928: 1.00488, 929: 1.00482, 930: 1.00502, 931: 1.00505, 932: 1.00488, 933: 1.00495, 934: 1.0049, 935: 1.00487, 936: 1.00473, 937: 1.00482, 938: 1.00477, 939: 1.00476, 940: 1.00476, 941: 1.00476, 942: 1.00475, 943: 1.0048, 944: 1.00488, 945: 1.00485, 946: 1.00487, 947: 1.0049, 948: 1.00476, 949: 1.0048, 950: 1.00476, 951: 1.00485, 952: 1.00483, 953: 1.00497, 954: 1.00493, 955: 1.00483, 956: 1.00475, 957: 1.00475, 958: 1.00479, 959: 1.00474, 960: 1.00475, 961: 1.00482, 962: 1.00474, 963: 1.00465, 964: 1.00456, 965: 1.00457, 966: 1.00449, 967: 1.00458, 968: 1.00455, 969: 1.00462, 970: 1.00454, 971: 1.00455, 972: 1.00455, 973: 1.00451, 974: 1.00463, 975: 1.00466, 976: 1.00463, 977: 1.00464, 978: 1.00458, 979: 1.00468, 980: 1.00465, 981: 1.00461, 982: 1.00465, 983: 1.00456, 984: 1.00459, 985: 1.00449, 986: 1.00454, 987: 1.00456, 988: 1.00443, 989: 1.00445, 990: 1.0045, 991: 1.00443, 992: 1.00441, 993: 1.00441, 994: 1.00445, 995: 1.00434, 996: 1.00448, 997: 1.00441, 998: 1.00442, 999: 1.00442, 1000: 1.00442, 1001: 1.00438, 1002: 1.00433, 1003: 1.00441, 1004: 1.00441, 1005: 1.00445, 1006: 1.00447, 1007: 1.00445, 1008: 1.00445, 1009: 1.0044, 1010: 1.0043, 1011: 1.00424, 1012: 1.00407, 1013: 1.00407, 1014: 1.00426, 1015: 1.00429, 1016: 1.00436, 1017: 1.00444, 1018: 1.0042, 1019: 1.00403, 1020: 1.00435, 1021: 1.00443, 1022: 1.00441, 1023: 1.00442, 1024: 1.00443, 1025: 1.00429, 1026: 1.00426, 1027: 1.00428, 1028: 1.00434, 1029: 1.00435, 1030: 1.00429, 1031: 1.00425, 1032: 1.00425, 1033: 1.00263, 1034: 1.00413, 1035: 1.00427, 1036: 1.00433, 1037: 1.00434, 1038: 1.00436, 1039: 1.00417, 1040: 1.00432, 1041: 1.00434, 1042: 1.00432, 1043: 1.00431, 1044: 1.00431, 1045: 1.00426, 1046: 1.00415, 1047: 1.00253, 1048: 1.00256, 1049: 1.00414, 1050: 1.00426, 1051: 1.00406, 1052: 1.00409, 1053: 1.00425, 1054: 1.00418, 1055: 1.00423, 1056: 1.00426, 1057: 1.00433, 1058: 1.00433, 1059: 1.00424, 1060: 1.00438, 1061: 1.00428, 1062: 1.00417, 1063: 1.00412, 1064: 1.00428, 1065: 1.00426, 1066: 1.00421, 1067: 1.00412, 1068: 1.00416, 1069: 1.00415, 1070: 1.00416, 1071: 1.00426, 1072: 1.00417, 1073: 1.00426, 1074: 1.0043, 1075: 1.00431, 1076: 1.00426, 1077: 1.00419, 1078: 1.00424, 1079: 1.00417, 1080: 1.00415, 1081: 1.00417, 1082: 1.00411, 1083: 1.00414, 1084: 1.00416, 1085: 1.00416, 1086: 1.00424, 1087: 1.00428, 1088: 1.00424, 1089: 1.00422, 1090: 1.00421, 1091: 1.00415, 1092: 1.00424, 1093: 1.00424, 1094: 1.00423, 1095: 1.00424, 1096: 1.00403, 1097: 1.00408, 1098: 1.00413, 1099: 1.00412, 1100: 1.00412, 1101: 1.00415, 1102: 1.00421, 1103: 1.00421, 1104: 1.00417, 1105: 1.00414, 1106: 1.00417, 1107: 1.00411, 1108: 1.00408, 1109: 1.00409, 1110: 1.00415, 1111: 1.00413, 1112: 1.00408, 1113: 1.00411, 1114: 1.00403, 1115: 1.00407, 1116: 1.00402, 1117: 1.00406, 1118: 1.00402, 1119: 1.00389, 1120: 1.00394, 1121: 1.00399, 1122: 1.00402, 1123: 1.00411, 1124: 1.00399, 1125: 1.00394, 1126: 1.00399, 1127: 1.00393, 1128: 1.00236, 1129: 1.00237, 1130: 1.00235, 1131: 1.00395, 1132: 1.00391, 1133: 1.00264, 1134: 1.00391, 1135: 1.00391, 1136: 1.00391, 1137: 1.00256, 1138: 1.00239, 1139: 1.0039, 1140: 1.00393, 1141: 1.00402, 1142: 1.00402, 1143: 1.00398, 1144: 1.00398, 1145: 1.00403, 1146: 1.00398, 1147: 1.00408, 1148: 1.00402, 1149: 1.00404, 1150: 1.00394, 1151: 1.00393, 1152: 1.00391, 1153: 1.00394, 1154: 1.00402, 1155: 1.00397, 1156: 1.00402, 1157: 1.00402, 1158: 1.004, 1159: 1.004, 1160: 1.00393, 1161: 1.00393, 1162: 1.0026, 1163: 1.00381, 1164: 1.00389, 1165: 1.00389, 1166: 1.004, 1167: 1.004, 1168: 1.004, 1169: 1.00386, 1170: 1.00397, 1171: 1.00396, 1172: 1.00396, 1173: 1.00397, 1174: 1.00397, 1175: 1.00394, 1176: 1.00394, 1177: 1.00396, 1178: 1.00397, 1179: 1.00403, 1180: 1.00399, 1181: 1.00398, 1182: 1.00404, 1183: 1.00406, 1184: 1.00399, 1185: 1.00389, 1186: 1.00387, 1187: 1.00385, 1188: 1.00378, 1189: 1.00385, 1190: 1.00234, 1191: 1.00374, 1192: 1.00387, 1193: 1.0039, 1194: 1.00385, 1195: 1.00236, 1196: 1.00377, 1197: 1.00385, 1198: 1.00384, 1199: 1.00381, 1200: 1.0024, 1201: 1.00376, 1202: 1.00236, 1203: 1.00379, 1204: 1.0038, 1205: 1.00357, 1206: 1.00233, 1207: 1.00371, 1208: 1.00371, 1209: 1.00378, 1210: 1.00384, 1211: 1.00374, 1212: 1.00394, 1213: 1.00392, 1214: 1.00384, 1215: 1.00377, 1216: 1.00356, 1217: 1.00223, 1218: 1.0022, 1219: 1.00218, 1220: 1.00362, 1221: 1.00367, 1222: 1.00377, 1223: 1.00367, 1224: 1.00373, 1225: 1.0037, 1226: 1.00376, 1227: 1.00377, 1228: 1.00352, 1229: 1.00244, 1230: 1.00211, 1231: 1.00243, 1232: 1.00213, 1233: 1.00211, 1234: 1.00217, 1235: 1.00218, 1236: 1.00225, 1237: 1.0023, 1238: 1.00377, 1239: 1.00382, 1240: 1.00365, 1241: 1.00375, 1242: 1.00376, 1243: 1.00235, 1244: 1.00371, 1245: 1.00377, 1246: 1.00365, 1247: 1.00375, 1248: 1.00217, 1249: 1.00208, 1250: 1.00217, 1251: 1.00231, 1252: 1.00227, 1253: 1.00234, 1254: 1.00227, 1255: 1.00209, 1256: 1.00209, 1257: 1.00208, 1258: 1.00208, 1259: 1.00208, 1260: 1.00208, 1261: 1.00204, 1262: 1.00204, 1263: 1.00204, 1264: 1.00202, 1265: 1.00198, 1266: 1.00204, 1267: 1.00207, 1268: 1.00204, 1269: 1.00206, 1270: 1.00204, 1271: 1.00207, 1272: 1.00206, 1273: 1.00207, 1274: 1.00209, 1275: 1.00207, 1276: 1.00206, 1277: 1.00208, 1278: 1.00205, 1279: 1.00204, 1280: 1.00207, 1281: 1.0021, 1282: 1.00209, 1283: 1.00209, 1284: 1.00206, 1285: 1.00203, 1286: 1.00207, 1287: 1.0021, 1288: 1.00202, 1289: 1.00209, 1290: 1.00207, 1291: 1.00211, 1292: 1.00211, 1293: 1.00211, 1294: 1.00209, 1295: 1.00208, 1296: 1.00206, 1297: 1.00204, 1298: 1.00208, 1299: 1.00207, 1300: 1.00199, 1301: 1.00196, 1302: 1.00199, 1303: 1.00196, 1304: 1.00198, 1305: 1.00202, 1306: 1.00198, 1307: 1.00198, 1308: 1.00203, 1309: 1.00203, 1310: 1.00204, 1311: 1.00198, 1312: 1.00198, 1313: 1.00191, 1314: 1.00189, 1315: 1.00195, 1316: 1.00199, 1317: 1.00199, 1318: 1.00199, 1319: 1.00199, 1320: 1.00204, 1321: 1.00202, 1322: 1.00198, 1323: 1.00198, 1324: 1.00202, 1325: 1.00202, 1326: 1.00202, 1327: 1.00202, 1328: 1.00198, 1329: 1.00202, 1330: 1.00196, 1331: 1.00197, 1332: 1.00197, 1333: 1.00194, 1334: 1.00189, 1335: 1.0019, 1336: 1.00194, 1337: 1.00197, 1338: 1.00202, 1339: 1.00196, 1340: 1.00193, 1341: 1.00195, 1342: 1.00195, 1343: 1.00193, 1344: 1.00191, 1345: 1.00192, 1346: 1.00191, 1347: 1.00188, 1348: 1.00185, 1349: 1.00186, 1350: 1.00186, 1351: 1.00186, 1352: 1.00186, 1353: 1.00186, 1354: 1.00185, 1355: 1.00185, 1356: 1.00186, 1357: 1.00186, 1358: 1.00184, 1359: 1.00182, 1360: 1.00186, 1361: 1.00185, 1362: 1.00186, 1363: 1.00186, 1364: 1.00188, 1365: 1.00191, 1366: 1.0019, 1367: 1.0019, 1368: 1.00193, 1369: 1.00192, 1370: 1.00191, 1371: 1.00192, 1372: 1.00191, 1373: 1.0019, 1374: 1.00191, 1375: 1.0019, 1376: 1.00189, 1377: 1.00188, 1378: 1.0019, 1379: 1.0019, 1380: 1.00189, 1381: 1.00188, 1382: 1.00188, 1383: 1.00188, 1384: 1.00189, 1385: 1.00191, 1386: 1.00189, 1387: 1.00188, 1388: 1.00189, 1389: 1.0019, 1390: 1.0019, 1391: 1.00191, 1392: 1.00193, 1393: 1.00194, 1394: 1.00192, 1395: 1.0019, 1396: 1.0019, 1397: 1.00189, 1398: 1.00188, 1399: 1.00187, 1400: 1.00186, 1401: 1.00186, 1402: 1.00186, 1403: 1.00186, 1404: 1.00187, 1405: 1.00188, 1406: 1.00186, 1407: 1.00186, 1408: 1.00183, 1409: 1.0018, 1410: 1.00183, 1411: 1.00181, 1412: 1.00184, 1413: 1.00186, 1414: 1.00188, 1415: 1.00188, 1416: 1.00188, 1417: 1.00188, 1418: 1.00188, 1419: 1.00185, 1420: 1.00185, 1421: 1.00186, 1422: 1.00186, 1423: 1.00187, 1424: 1.00187, 1425: 1.00185, 1426: 1.00183, 1427: 1.00182, 1428: 1.00182, 1429: 1.0018, 1430: 1.0018, 1431: 1.00181, 1432: 1.0018, 1433: 1.00174, 1434: 1.00174, 1435: 1.00177, 1436: 1.00179, 1437: 1.00177, 1438: 1.00179, 1439: 1.00178, 1440: 1.00177, 1441: 1.00177, 1442: 1.00179, 1443: 1.00178, 1444: 1.00178, 1445: 1.00179, 1446: 1.0018, 1447: 1.00176, 1448: 1.00179, 1449: 1.00178, 1450: 1.00179, 1451: 1.0018, 1452: 1.00179, 1453: 1.00178, 1454: 1.00177, 1455: 1.00179, 1456: 1.0018, 1457: 1.0018, 1458: 1.00179, 1459: 1.00177, 1460: 1.00179, 1461: 1.00179, 1462: 1.00177, 1463: 1.00175, 1464: 1.00175, 1465: 1.00179, 1466: 1.00177, 1467: 1.00177, 1468: 1.00176, 1469: 1.00175, 1470: 1.00173, 1471: 1.00175, 1472: 1.00175, 1473: 1.00175, 1474: 1.00174, 1475: 1.00174, 1476: 1.00175, 1477: 1.00175, 1478: 1.00174, 1479: 1.00174, 1480: 1.00174, 1481: 1.00174, 1482: 1.00174, 1483: 1.00174, 1484: 1.00174, 1485: 1.00175, 1486: 1.00174, 1487: 1.00173, 1488: 1.00171, 1489: 1.00171, 1490: 1.00168, 1491: 1.0017, 1492: 1.00174, 1493: 1.00174, 1494: 1.00174, 1495: 1.00175, 1496: 1.00175, 1497: 1.00175, 1498: 1.00174, 1499: 1.00175, 1500: 1.00175, 1501: 1.00174, 1502: 1.00175, 1503: 1.00175, 1504: 1.00174, 1505: 1.00172, 1506: 1.00171, 1507: 1.00172, 1508: 1.00175, 1509: 1.00176, 1510: 1.00177, 1511: 1.00176, 1512: 1.00173, 1513: 1.00171, 1514: 1.00171, 1515: 1.00171, 1516: 1.00172, 1517: 1.00172, 1518: 1.00172, 1519: 1.00171, 1520: 1.00172, 1521: 1.00171, 1522: 1.00172, 1523: 1.00172, 1524: 1.00173, 1525: 1.00174, 1526: 1.00174, 1527: 1.00173, 1528: 1.00175, 1529: 1.00171, 1530: 1.00171, 1531: 1.00172, 1532: 1.0017, 1533: 1.00173, 1534: 1.00173, 1535: 1.00171, 1536: 1.00171, 1537: 1.00171, 1538: 1.00171, 1539: 1.00172, 1540: 1.00172, 1541: 1.00174, 1542: 1.00171, 1543: 1.00171, 1544: 1.00172, 1545: 1.00175, 1546: 1.00172, 1547: 1.00172, 1548: 1.00172, 1549: 1.00171, 1550: 1.00169, 1551: 1.00169, 1552: 1.0017, 1553: 1.00173, 1554: 1.00172, 1555: 1.00173, 1556: 1.00173, 1557: 1.0017, 1558: 1.00171, 1559: 1.00172, 1560: 1.00169, 1561: 1.00169, 1562: 1.00166, 1563: 1.00167, 1564: 1.00169, 1565: 1.00168, 1566: 1.00169, 1567: 1.00169, 1568: 1.00167, 1569: 1.00167, 1570: 1.00164, 1571: 1.00166, 1572: 1.00164, 1573: 1.00164, 1574: 1.00164, 1575: 1.00165, 1576: 1.00167, 1577: 1.00167, 1578: 1.00168, 1579: 1.00171, 1580: 1.00168, 1581: 1.00168, 1582: 1.00167, 1583: 1.00167, 1584: 1.00161, 1585: 1.00162, 1586: 1.00159, 1587: 1.00164, 1588: 1.00164, 1589: 1.00165, 1590: 1.00166, 1591: 1.00166, 1592: 1.00166, 1593: 1.00166, 1594: 1.00167, 1595: 1.00169, 1596: 1.00169, 1597: 1.00172, 1598: 1.0017, 1599: 1.0017, 1600: 1.0017, 1601: 1.0017, 1602: 1.00167, 1603: 1.00167, 1604: 1.00166, 1605: 1.00166, 1606: 1.00165, 1607: 1.00162, 1608: 1.00166, 1609: 1.00166, 1610: 1.00167, 1611: 1.00167, 1612: 1.00167, 1613: 1.00165, 1614: 1.00161, 1615: 1.00153, 1616: 1.00162, 1617: 1.00164, 1618: 1.00162, 1619: 1.00157, 1620: 1.00163, 1621: 1.00163, 1622: 1.0016, 1623: 1.00159, 1624: 1.0016, 1625: 1.0, 1626: 1.00162, 1627: 1.00161, 1628: 1.00163, 1629: 1.00162, 1630: 1.0016, 1631: 1.00162, 1632: 1.00162, 1633: 1.00162, 1634: 1.00164, 1635: 1.00166, 1636: 1.00162, 1637: 1.00162, 1638: 1.00162, 1639: 1.00168, 1640: 1.00162, 1641: 1.00165, 1642: 1.00165, 1643: 1.00163, 1644: 1.00162, 1645: 1.00163, 1646: 1.00162, 1647: 1.00162, 1648: 1.00164, 1649: 1.00166, 1650: 1.00166, 1651: 1.00165, 1652: 1.00167, 1653: 1.00167, 1654: 1.00166, 1655: 1.00165, 1656: 1.00163, 1657: 1.00162, 1658: 1.00161, 1659: 1.00162, 1660: 1.00165, 1661: 1.00163, 1662: 1.00163, 1663: 1.00163, 1664: 1.00161, 1665: 1.00165, 1666: 1.00165, 1667: 1.00165, 1668: 1.0016, 1669: 1.0016, 1670: 1.0016, 1671: 1.0016, 1672: 1.00165, 1673: 1.00164, 1674: 1.00165, 1675: 1.00163, 1676: 1.00166, 1677: 1.00163, 1678: 1.00163, 1679: 1.00163, 1680: 1.00162, 1681: 1.0016, 1682: 1.00159, 1683: 1.00157, 1684: 1.00155, 1685: 1.0, 1686: 1.0, 1687: 1.0, 1688: 1.0, 1689: 1.0, 1690: 1.0, 1691: 1.00151, 1692: 1.00158, 1693: 1.00156, 1694: 1.00154, 1695: 1.0015, 1696: 1.00158, 1697: 1.00161, 1698: 1.00164, 1699: 1.00162, 1700: 1.00146, 1701: 1.00161, 1702: 1.00157, 1703: 1.00157, 1704: 1.00152, 1705: 1.00152, 1706: 1.00149, 1707: 1.00149, 1708: 1.0, 1709: 1.0, 1710: 1.0, 1711: 1.00144, 1712: 1.0, 1713: 1.00148, 1714: 1.00153, 1715: 1.00157, 1716: 1.00157, 1717: 1.00155, 1718: 1.00152, 1719: 1.00144, 1720: 1.0, 1721: 1.0, 1722: 1.00156, 1723: 1.00148, 1724: 1.00144, 1725: 1.00149, 1726: 1.0, 1727: 1.0, 1728: 1.00149, 1729: 1.00156, 1730: 1.00156, 1731: 1.00159, 1732: 1.00161, 1733: 1.00159, 1734: 1.00162, 1735: 1.00159, 1736: 1.00162, 1737: 1.00162, 1738: 1.00159, 1739: 1.00159, 1740: 1.00162, 1741: 1.00162, 1742: 1.00158, 1743: 1.00159, 1744: 1.00157, 1745: 1.00156, 1746: 1.00155, 1747: 1.00155, 1748: 1.00155, 1749: 1.00158, 1750: 1.00159, 1751: 1.00159, 1752: 1.00158, 1753: 1.00156, 1754: 1.00159, 1755: 1.00159, 1756: 1.00155, 1757: 1.00155, 1758: 1.00156, 1759: 1.00158, 1760: 1.00156, 1761: 1.00154, 1762: 1.00152, 1763: 1.00152, 1764: 1.00152, 1765: 1.0, 1766: 1.0, 1767: 1.0, 1768: 1.0, 1769: 1.0, 1770: 1.0, 1771: 1.0, 1772: 1.0, 1773: 1.0, 1774: 1.0, 1775: 1.0, 1776: 1.0, 1777: 1.0, 1778: 1.0, 1779: 1.0, 1780: 1.0, 1781: 1.0, 1782: 1.0, 1783: 1.0, 1784: 1.0, 1785: 1.0, 1786: 1.0, 1787: 1.0, 1788: 1.0, 1789: 1.0, 1790: 1.0, 1791: 1.0, 1792: 1.0, 1793: 1.0, 1794: 1.0, 1795: 1.0, 1796: 1.0, 1797: 1.0, 1798: 1.0, 1799: 1.0, 1800: 1.0, 1801: 1.0, 1802: 1.0, 1803: 1.0, 1804: 1.0, 1805: 1.0, 1806: 1.0, 1807: 1.0, 1808: 1.0, 1809: 1.0, 1810: 1.0, 1811: 1.0, 1812: 1.0, 1813: 1.0, 1814: 1.0, 1815: 1.0, 1816: 1.00153, 1817: 1.0015, 1818: 1.0, 1819: 1.00136, 1820: 1.00151, 1821: 1.0, 1822: 1.0, 1823: 1.0, 1824: 1.0, 1825: 1.0, 1826: 1.0, 1827: 1.00141, 1828: 1.00147, 1829: 1.00147, 1830: 1.0, 1831: 1.0, 1832: 1.0, 1833: 1.0, 1834: 1.0, 1835: 1.0, 1836: 1.0, 1837: 1.0, 1838: 1.0, 1839: 1.0, 1840: 1.0, 1841: 1.0, 1842: 1.0, 1843: 1.0, 1844: 1.0, 1845: 1.00148, 1846: 1.00149, 1847: 1.0, 1848: 1.0, 1849: 1.0, 1850: 1.0, 1851: 1.0, 1852: 1.0, 1853: 1.0, 1854: 1.0, 1855: 1.0, 1856: 1.0, 1857: 1.0, 1858: 1.0, 1859: 1.0, 1860: 1.0, 1861: 1.0, 1862: 1.0, 1863: 1.0, 1864: 1.0, 1865: 1.0, 1866: 1.0, 1867: 1.0, 1868: 1.0, 1869: 1.0, 1870: 1.0, 1871: 1.0, 1872: 1.0, 1873: 1.0, 1874: 1.0, 1875: 1.0, 1876: 1.0, 1877: 1.0, 1878: 1.0, 1879: 1.0, 1880: 1.0, 1881: 1.0, 1882: 1.0, 1883: 1.0, 1884: 1.0, 1885: 1.0, 1886: 1.0, 1887: 1.0, 1888: 1.0, 1889: 1.0, 1890: 1.0, 1891: 1.0, 1892: 1.0, 1893: 1.0, 1894: 1.0, 1895: 1.0, 1896: 1.0, 1897: 1.0, 1898: 1.0, 1899: 1.0, 1900: 1.0, 1901: 1.0, 1902: 1.0, 1903: 1.0, 1904: 1.0, 1905: 1.0, 1906: 1.0, 1907: 1.0, 1908: 1.0, 1909: 1.0, 1910: 1.0, 1911: 1.0, 1912: 1.0, 1913: 1.0, 1914: 1.0, 1915: 1.0, 1916: 1.0, 1917: 1.0, 1918: 1.0, 1919: 1.0, 1920: 1.0, 1921: 1.0, 1922: 1.0, 1923: 1.0, 1924: 1.0, 1925: 1.0, 1926: 1.0, 1927: 1.0, 1928: 1.0, 1929: 1.0, 1930: 1.0, 1931: 1.0, 1932: 1.0, 1933: 1.0, 1934: 1.0, 1935: 1.0, 1936: 1.0, 1937: 1.0, 1938: 1.0, 1939: 1.0, 1940: 1.0, 1941: 1.0, 1942: 1.0, 1943: 1.0, 1944: 1.0, 1945: 1.0, 1946: 1.0, 1947: 1.0, 1948: 1.0, 1949: 1.0, 1950: 1.0, 1951: 1.0, 1952: 1.0, 1953: 1.0, 1954: 1.0, 1955: 1.0, 1956: 1.0, 1957: 1.0, 1958: 1.0, 1959: 1.0, 1960: 1.0, 1961: 1.0, 1962: 1.0, 1963: 1.0, 1964: 1.0, 1965: 1.0, 1966: 1.0, 1967: 1.0, 1968: 1.0, 1969: 1.0, 1970: 1.0, 1971: 1.0, 1972: 1.0, 1973: 1.0, 1974: 1.0, 1975: 1.0, 1976: 1.0, 1977: 1.0, 1978: 1.0, 1979: 1.0, 1980: 1.00131, 1981: 1.0, 1982: 1.0, 1983: 1.0, 1984: 1.0, 1985: 1.0, 1986: 1.0, 1987: 1.0, 1988: 1.0, 1989: 1.0, 1990: 1.0, 1991: 1.0, 1992: 1.0, 1993: 1.0, 1994: 1.0, 1995: 1.0, 1996: 1.0, 1997: 1.0, 1998: 1.0, 1999: 1.0}    
        if step < buffer:
            return base_variance
    #     print('50_step_score:{}, current_score:{}'.format(prev_score_50, current_score_50))
        
        score_change = self.score_track[-1] / self.score_track[-window-1]
        if score_change >= perc_50[step] and self.increase_counter > -max_counter:
            gap = perc_97[step] - perc_50[step]
            diff = score_change - perc_50[step]
            ratio_power = max([diff/gap,0])**power
            variance = base_variance * (1- reduc_variance * min([ratio_power,1.0]))
            if ratio_power >= 0.5:
                self.increase_counter -= 1
        
        elif score_change < perc_50[step] and self.increase_counter < max_counter:
            gap = perc_50[step] - perc_3[step]
            diff = perc_50[step] - score_change
            ratio_power = max([diff/gap,0])**power
            variance = base_variance + additional_variance * min([ratio_power,1.0])
            if ratio_power >= 0.5:
                self.increase_counter += 1
        else:
            return base_variance
    #     print('step:{}, variance:{}, score_change:{}, median:{}, 97th_perc:{}, 3rd_perc:{}'\
    #           .format(step, variance, score_change, perc_50[step], perc_97[step], perc_3[step]))
        if self.increase_counter == round(max_counter*0.95,0):
            print('reaching max_counter')
        return variance
        
    def action(self, observation, configuration):  
        assert len(self.player_track) == len(self.enemy_track)
        assert len(self.player_track) == observation.step
        base_enemy_focus = 0.0
        bonus_enemy_focus = 10.0
        enemy_grad = 2.0
        player_decay = 0.03
        enemy_decay = 0.00
        base_variance = 1.0 # variance_weight
        additional_variance = 0.5
        reduc_variance = 0.8
        follow_window = 50
        unexplored_c = 0.5
        spy_window = 50
    
        step_num = observation.step
                                        
        if step_num == 0:
            variance = base_variance
            total_decay = 0
        else:
            variance = self.get_variance(base_variance, reduc_variance,  additional_variance, step_num)
        
        #Adding in decay term
        enemy_hist_full = self.create_hist(self.enemy_track)
        player_decay = (1-player_decay)**(self.post_a+self.post_b-2) 
        enemy_decay = (1-enemy_decay) ** (enemy_hist_full)
        total_decay = player_decay * enemy_decay
            
        #calculating enemy constant
        enemy_comp = self.enemy_competence()
        enemy_focus = base_enemy_focus + bonus_enemy_focus * enemy_comp
        enemy_hist = self.create_hist(self.enemy_track[-spy_window:])
        z = - (enemy_grad*enemy_hist - 4) #shift right by 3 units to achieve smooth curve starting from 0.1
        enemy_corr = enemy_focus / (1 + np.exp(z))

        
        #Making choice
        unexplored = self.unexplored_bandits() * unexplored_c
        post_a_decayed = self.post_a * total_decay 
        bandit_var = beta.std(post_a_decayed, self.post_b) * variance
        decayed_prior = (post_a_decayed + enemy_corr + unexplored) / (post_a_decayed + self.post_b) 
        bound = decayed_prior + bandit_var
        
        
        top_choice = self.random_top_choice(bound, std_limit = 0.0)
        
        #detect strict followers
        if step_num >= follow_window and self.cont_second_choices <= follow_window/1.5:
            #if being followed, use alternating strategy
            follow_score = self.detect_follower(lookback = 3, window = follow_window)
            bandit = self.alternate_choice(top_choice, follow_score=follow_score, bound=bound)
        else:
            bandit = top_choice
            
        self.bandit = bandit
    
    
#        print("step: {}, score:{}, bandit:{}, decayed_prior:{}, enemy_focus: {},enemy_corr:{}, bandit_var:{}, var_param:{},"\
#               .format(step_num, observation.reward, bandit, round(decayed_prior[bandit],3),enemy_focus, round(enemy_corr[bandit],3),\
#                       round(bandit_var[bandit],3), round(variance,3)))
    
#     if step_num in [100, 200, 400, 700, 1000, 1500, 1998]:
#         print("Step_Count: {}, total_reward: {},  variance:{}"\
#               .format(step_num, total_reward, variance))
        if step_num == 1998:
            print("second_choices:{}".format(self.second_choices))
        return bandit
        
agent = BayesianSpy()
def execute_agent(observation, configuration):
    return agent(observation, configuration)